### In this script, DBN is run on the individual dates
### In this script, DBN is run with 1s time bin, 3 time lag 
### In this script, the animal tracking is done with only one camera - camera 2 (middle) 
### This script analyze the autolever sessions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn
import scipy
import scipy.stats as st
import sklearn
from sklearn.neighbors import KernelDensity
import string
import warnings
import pickle

import os
import glob
import random
from time import time

from pgmpy.models import BayesianModel
from pgmpy.models import DynamicBayesianNetwork as DBN
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch,BicScore
from pgmpy.base import DAG
import networkx as nx


### function - get body part location for each pair of cameras

In [ ]:
from ana_functions.body_part_locs_eachpair import body_part_locs_eachpair
from ana_functions.body_part_locs_singlecam import body_part_locs_singlecam

### function - align the two cameras

In [ ]:
from ana_functions.camera_align import camera_align       

### function - merge the two pairs of cameras

In [ ]:
from ana_functions.camera_merge import camera_merge

### function - find social gaze time point

In [ ]:
from ana_functions.find_socialgaze_timepoint import find_socialgaze_timepoint
from ana_functions.find_socialgaze_timepoint_singlecam import find_socialgaze_timepoint_singlecam
from ana_functions.find_socialgaze_timepoint_singlecam_wholebody import find_socialgaze_timepoint_singlecam_wholebody
from ana_functions.find_socialgaze_timepoint_singlecam_wholebody_2 import find_socialgaze_timepoint_singlecam_wholebody_2


### function - define time point of behavioral events

In [ ]:
from ana_functions.bhv_events_timepoint import bhv_events_timepoint
from ana_functions.bhv_events_timepoint_singlecam import bhv_events_timepoint_singlecam
from ana_functions.bhv_events_timepoint_singlecam_otherlever import bhv_events_timepoint_singlecam_otherlever

### function - plot behavioral events

In [ ]:
from ana_functions.plot_bhv_events import plot_bhv_events
from ana_functions.plot_bhv_events_levertube import plot_bhv_events_levertube
from ana_functions.draw_self_loop import draw_self_loop
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection
from ana_functions.plot_gaze_along_phase_of_continuous_bhv_var_singlecam import plot_gaze_along_phase_of_continuous_bhv_var_singlecam


### function - make demo videos with skeleton and inportant vectors

In [ ]:
from ana_functions.tracking_video_singlecam_demo import tracking_video_singlecam_demo
from ana_functions.tracking_video_singlecam_wholebody_demo import tracking_video_singlecam_wholebody_demo

### function - interval between all behavioral events

In [ ]:
from ana_functions.bhv_events_interval import bhv_events_interval
from ana_functions.bhv_events_interval import bhv_events_interval_certainEdges

### function - train the dynamic bayesian network - multi time lag (3 lags)

In [ ]:
from ana_functions.train_DBN_multiLag_autolever import train_DBN_multiLag
from ana_functions.train_DBN_multiLag_autolever import train_DBN_multiLag_create_df_only
from ana_functions.train_DBN_multiLag_autolever import train_DBN_multiLag_training_only
from ana_functions.train_DBN_multiLag_autolever import graph_to_matrix
from ana_functions.train_DBN_multiLag_autolever import get_weighted_dags
from ana_functions.train_DBN_multiLag_autolever import get_significant_edges
from ana_functions.train_DBN_multiLag_autolever import threshold_edges
from ana_functions.train_DBN_multiLag_autolever import Modulation_Index
from ana_functions.EfficientTimeShuffling import EfficientShuffle
from ana_functions.AicScore import AicScore

## Analyze each session

### prepare the basic behavioral data (especially the time stamps for each bhv events)

In [ ]:
# instead of using gaze angle threshold, use the target rectagon to deside gaze info
# ...need to update
sqr_thres_tubelever = 75 # draw the square around tube and lever
sqr_thres_face = 1.15 # a ratio for defining face boundary
sqr_thres_body = 4 # how many times to enlongate the face box boundry to the body


# get the fps of the analyzed video
fps = 30

# frame number of the demo video
# nframes = 0.5*30 # second*30fps
nframes = 2*30 # second*30fps

# re-analyze the video or not
reanalyze_video = 0
redo_anystep = 0

# only analyze the best (five) sessions for each conditions
do_bestsession = 1
if do_bestsession:
    savefile_sufix = '_bestsessions_autolever'
else:
    savefile_sufix = '_autolever'
    
# all the videos (no misaligned ones)
# aligned with the audio
# get the session start time from "videosound_bhv_sync.py/.ipynb"
# currently the session_start_time will be manually typed in. It can be updated after a better method is used

# dodson scorch
if 1:
    if not do_bestsession:
        dates_list = [                      
                     ]
        session_start_times = [                                
                              ] # in second
    elif do_bestsession:
        # pick only five sessions for each conditions
        dates_list = [        
                       "20220915","20220920","20221010","20230208",
            
                       "20221205","20221206","20221209","20221214","20230112",
                      
                      "20230327","20230328","20230331","20230403","20230404",
                      "20230405","20230406",
                     ]
        session_start_times = [                 
                                 0.00,33.03, 6.50, 0.00, 
            
                                 0.00, 0.00, 21.7, 17.0, 14.2, 
            
                                 30.0, 34.8, 29.2, 27.0, 28.0,
                                 26.5, 23.0,                                
                              ] # in second
    
    animal1_fixedorder = ['dodson']
    animal2_fixedorder = ['scorch']

    animal1_filename = "Dodson"
    animal2_filename = "Scorch"
    
# eddie sparkle
if 0:
    if not do_bestsession:
        dates_list = [                                            
                   ]
        session_start_times = [                                                                 
                              ] # in second
    elif do_bestsession:   
        dates_list = [
                      "20221122",  "20221125",  
                      
                      "20230321",  "20230322",  "20230324",  "20230327",  "20230328",
            
                      "20230524",  "20230526",  "20230529",  "20230531",  "20230605", 
                      "20230606",  "20230608",  "20230609",  "20230613",  "20230615",
                   ]
        session_start_times = [ 
                                  8.00,  38.00, 
                                  
                                  7.00,   7.50,  5.50, 11.00,  9.00,
                            
                                  25.0,   34.0,  29.0,  12.8,  26.0,  
                                  24.1,   21.0,  23.0,  29.0,  24.5,
                              ] # in second
    
    animal1_fixedorder = ['eddie']
    animal2_fixedorder = ['sparkle']

    animal1_filename = "Eddie"
    animal2_filename = "Sparkle"
    
# ginger kanga
if 0:
    if not do_bestsession:
        dates_list = [                     
                   ]
        session_start_times = [                            
                              ] # in second 
    elif do_bestsession:   
        dates_list = [
                      "20230214","20230216",

                      "20230412","20230413","20230517",
            
                      "20230616","20230620","20230621","20230622","20230623",
                      "20230626","20230627","20230629","20230703",                    
                   ]
        session_start_times = [ 
                                 0.00, 48.00, 

                                 0.00,  0.00,  0.00,  
            
                                58.20, 60.20, 26.80, 43.50, 40.00,
                                53.00, 51.00, 27.20, 32.20,
                                
                              ] # in second 
    
    animal1_fixedorder = ['ginger']
    animal2_fixedorder = ['kanga']

    animal1_filename = "Ginger"
    animal2_filename = "Kanga"

    
# dannon kanga
if 0:
    if not do_bestsession:
        dates_list = [
                   ]
        session_start_times = [ 
                              ] # in second 
    elif do_bestsession:   
        dates_list = [
                      "20230718","20230720","20230914",
            
                      "20230907","20230915","20230918","20230926","20230928",
                      "20231002","20231010",
            
                      "20240201","20240212","20240214","20240215","20240222",
                      "20240223","20240226",
            
                   ]
        session_start_times = [ 
                                    0,     0,     0, 
            
                                    0,     0,     0,     0,     0,     
                                    0,     0,
            
                                    21.00, 45.50, 19.00, 22.20, 28.20,
                                    25.50, 16.10,
                              ] # in second 
    
    animal1_fixedorder = ['dannon']
    animal2_fixedorder = ['kanga']

    animal1_filename = "Dannon"
    animal2_filename = "Kanga"
    
    
    
#    
# dates_list = ["20230718"]
# session_start_times = [0.00] # in second

ndates = np.shape(dates_list)[0]

session_start_frames = session_start_times * fps # fps is 30Hz

totalsess_time = 600

# video tracking results info
animalnames_videotrack = ['dodson','scorch'] # does not really mean dodson and scorch, instead, indicate animal1 and animal2
bodypartnames_videotrack = ['rightTuft','whiteBlaze','leftTuft','rightEye','leftEye','mouth']


# which camera to analyzed
cameraID = 'camera-2'
cameraID_short = 'cam2'


# location of levers and tubes for camera 2
# get this information using DLC animal tracking GUI, the results are stored: 
# /home/ws523/marmoset_tracking_DLCv2/marmoset_tracking_with_lever_tube-weikang-2023-04-13/labeled-data/
considerlevertube = 1
considertubeonly = 0
# # camera 1
# lever_locs_camI = {'dodson':np.array([645,600]),'scorch':np.array([425,435])}
# tube_locs_camI  = {'dodson':np.array([1350,630]),'scorch':np.array([555,345])}
# # camera 2
lever_locs_camI = {'dodson':np.array([1335,715]),'scorch':np.array([550,715])}
tube_locs_camI  = {'dodson':np.array([1550,515]),'scorch':np.array([350,515])}
# # lever_locs_camI = {'dodson':np.array([1335,715]),'scorch':np.array([550,715])}
# # tube_locs_camI  = {'dodson':np.array([1650,490]),'scorch':np.array([250,490])}
# # camera 3
# lever_locs_camI = {'dodson':np.array([1580,440]),'scorch':np.array([1296,540])}
# tube_locs_camI  = {'dodson':np.array([1470,375]),'scorch':np.array([805,475])}


if np.shape(session_start_times)[0] != np.shape(dates_list)[0]:
    exit()

    
# define bhv events summarizing variables     
tasktypes_all_dates = np.zeros((ndates,1))
coopthres_all_dates = np.zeros((ndates,1))

succ_rate_all_dates = np.zeros((ndates,1))
interpullintv_all_dates = np.zeros((ndates,1))
trialnum_all_dates = np.zeros((ndates,1))

owgaze1_num_all_dates = np.zeros((ndates,1))
owgaze2_num_all_dates = np.zeros((ndates,1))
mtgaze1_num_all_dates = np.zeros((ndates,1))
mtgaze2_num_all_dates = np.zeros((ndates,1))
pull1_num_all_dates = np.zeros((ndates,1))
pull2_num_all_dates = np.zeros((ndates,1))

gazelever1_num_all_dates = np.zeros((ndates,1))
gazelever2_num_all_dates = np.zeros((ndates,1))
gazetube1_num_all_dates = np.zeros((ndates,1))
gazetube2_num_all_dates = np.zeros((ndates,1))

gazeotherlever1_num_all_dates = np.zeros((ndates,1))
gazeotherlever2_num_all_dates = np.zeros((ndates,1))

session_recordtime_all_dates = np.zeros((ndates,1))

bhv_intv_all_dates = dict.fromkeys(dates_list, [])
pull_edges_intv_all_dates = dict.fromkeys(dates_list, [])

gazeDist_phaseof_contbhvvar_all_dates = dict.fromkeys(dates_list, [])

# where to save the summarizing data
data_saved_folder = '/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/3d_recontruction_analysis_self_and_coop_task_data_saved/'


    

In [ ]:
# basic behavior analysis (define time stamps for each bhv events, etc)

try:
    if redo_anystep:
        dummy
    
    # load saved data
    data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    
    with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull2_num_all_dates = pickle.load(f)
        
    with open(data_saved_subfolder+'/gazelever1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazelever1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/gazelever2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazelever2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/gazetube1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazetube1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/gazetube2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazetube2_num_all_dates = pickle.load(f)
    
    with open(data_saved_subfolder+'/gazeotherlever1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazeotherlever1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/gazeotherlever2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazeotherlever2_num_all_dates = pickle.load(f)
    
    with open(data_saved_subfolder+'/session_recordtime_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        session_recordtime_all_dates = pickle.load(f)

    with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        tasktypes_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        coopthres_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        succ_rate_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        interpullintv_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        trialnum_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        bhv_intv_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull_edges_intv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull_edges_intv_all_dates = pickle.load(f)

    with open(data_saved_subfolder+'/gazeDist_phaseof_contbhvvar_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        gazeDist_phaseof_contbhvvar_all_dates = pickle.load(f)   

    print('all data from all dates are loaded')

except:

    print('analyze all dates')

    for idate in np.arange(0,ndates,1):
        date_tgt = dates_list[idate]
        session_start_time = session_start_times[idate]

        # folder and file path
        camera12_analyzed_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_camera12/"
        camera23_analyzed_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_camera23/"
        
        singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_cameraSep1shuffle1_150000"
        try: 
            bodyparts_camI_camIJ = camera12_analyzed_path+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            # get the bodypart data from files
            bodyparts_locs_camI = body_part_locs_singlecam(bodyparts_camI_camIJ,singlecam_ana_type,animalnames_videotrack,bodypartnames_videotrack,date_tgt)
            video_file_original = camera12_analyzed_path+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+".mp4"
        except:
            bodyparts_camI_camIJ = camera23_analyzed_path+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            # get the bodypart data from files
            bodyparts_locs_camI = body_part_locs_singlecam(bodyparts_camI_camIJ,singlecam_ana_type,animalnames_videotrack,bodypartnames_videotrack,date_tgt)
            video_file_original = camera23_analyzed_path+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+".mp4"        
        
        #
        recordedtime = np.shape(bodyparts_locs_camI[('dodson','rightTuft')])[0]/fps # - session_start_time
        session_recordtime_all_dates[idate] = recordedtime
        
        
        # load behavioral results
        try:
            try:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_from_task_code/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path +date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
            except:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_from_task_code/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
        except:
            try:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_forceManipulation_task/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path +date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
            except:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_forceManipulation_task/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])

        # get animal info from the session information
        animal1 = session_info['lever1_animal'][0].lower()
        animal2 = session_info['lever2_animal'][0].lower()

        
        # get task type and cooperation threshold
        try:
            coop_thres = session_info["pulltime_thres"][0]
            tasktype = session_info["task_type"][0]
        except:
            coop_thres = 0
            tasktype = 1
        tasktypes_all_dates[idate] = tasktype
        coopthres_all_dates[idate] = coop_thres   

        # clean up the trial_record
        warnings.filterwarnings('ignore')
        trial_record_clean = pd.DataFrame(columns=trial_record.columns)
        for itrial in np.arange(0,np.max(trial_record['trial_number']),1):
            # trial_record_clean.loc[itrial] = trial_record[trial_record['trial_number']==itrial+1].iloc[[0]]
            trial_record_clean = trial_record_clean.append(trial_record[trial_record['trial_number']==itrial+1].iloc[[0]])
        trial_record_clean = trial_record_clean.reset_index(drop = True)

        # change bhv_data time to the absolute time
        time_points_new = pd.DataFrame(np.zeros(np.shape(bhv_data)[0]),columns=["time_points_new"])
        for itrial in np.arange(0,np.max(trial_record_clean['trial_number']),1):
            ind = bhv_data["trial_number"]==itrial+1
            new_time_itrial = bhv_data[ind]["time_points"] + trial_record_clean["trial_starttime"].iloc[itrial]
            time_points_new["time_points_new"][ind] = new_time_itrial
        bhv_data["time_points"] = time_points_new["time_points_new"]
        bhv_data = bhv_data[bhv_data["time_points"] != 0]


        # analyze behavior results
        # succ_rate_all_dates[idate] = np.sum(trial_record_clean["rewarded"]>0)/np.shape(trial_record_clean)[0]
        succ_rate_all_dates[idate] = np.sum((bhv_data['behavior_events']==3)|(bhv_data['behavior_events']==4))/np.sum((bhv_data['behavior_events']==1)|(bhv_data['behavior_events']==2))
        trialnum_all_dates[idate] = np.shape(trial_record_clean)[0]
        #
        pullid = np.array(bhv_data[(bhv_data['behavior_events']==1) | (bhv_data['behavior_events']==2)]["behavior_events"])
        pulltime = np.array(bhv_data[(bhv_data['behavior_events']==1) | (bhv_data['behavior_events']==2)]["time_points"])
        pullid_diff = np.abs(pullid[1:] - pullid[0:-1])
        pulltime_diff = pulltime[1:] - pulltime[0:-1]
        interpull_intv = pulltime_diff[pullid_diff==1]
        interpull_intv = interpull_intv[interpull_intv<10]
        mean_interpull_intv = np.nanmean(interpull_intv)
        std_interpull_intv = np.nanstd(interpull_intv)
        #
        interpullintv_all_dates[idate] = mean_interpull_intv
        # 
        if np.isin(animal1,animal1_fixedorder):
            pull1_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==1) 
            pull2_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==2)
        else:
            pull1_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==2) 
            pull2_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==1)

        
        # load behavioral event results
        try:
            # dummy
            print('load social gaze with '+cameraID+' only of '+date_tgt)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_look_ornot.pkl', 'rb') as f:
                output_look_ornot = pickle.load(f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allvectors.pkl', 'rb') as f:
                output_allvectors = pickle.load(f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allangles.pkl', 'rb') as f:
                output_allangles = pickle.load(f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_key_locations.pkl', 'rb') as f:
                output_key_locations = pickle.load(f)
        except:   
            print('analyze social gaze with '+cameraID+' only of '+date_tgt)
            # get social gaze information 
            output_look_ornot, output_allvectors, output_allangles = find_socialgaze_timepoint_singlecam_wholebody(bodyparts_locs_camI,lever_locs_camI,tube_locs_camI,
                                                                                                                   considerlevertube,considertubeonly,sqr_thres_tubelever,
                                                                                                                   sqr_thres_face,sqr_thres_body)
            output_key_locations = find_socialgaze_timepoint_singlecam_wholebody_2(bodyparts_locs_camI,lever_locs_camI,tube_locs_camI,considerlevertube)
            
            # save data
            current_dir = data_saved_folder+'/bhv_events_singlecam_wholebody/'+animal1_fixedorder[0]+animal2_fixedorder[0]
            add_date_dir = os.path.join(current_dir,cameraID+'/'+date_tgt)
            if not os.path.exists(add_date_dir):
                os.makedirs(add_date_dir)
            #
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_look_ornot.pkl', 'wb') as f:
                pickle.dump(output_look_ornot, f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allvectors.pkl', 'wb') as f:
                pickle.dump(output_allvectors, f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allangles.pkl', 'wb') as f:
                pickle.dump(output_allangles, f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_key_locations.pkl', 'wb') as f:
                pickle.dump(output_key_locations, f)
  

        look_at_other_or_not_merge = output_look_ornot['look_at_other_or_not_merge']
        look_at_tube_or_not_merge = output_look_ornot['look_at_tube_or_not_merge']
        look_at_lever_or_not_merge = output_look_ornot['look_at_lever_or_not_merge']
        look_at_otherlever_or_not_merge = output_look_ornot['look_at_otherlever_or_not_merge']
        look_at_otherface_or_not_merge = output_look_ornot['look_at_otherface_or_not_merge']
        
        # change the unit to second
        session_start_time = session_start_times[idate]
        look_at_other_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_other_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_lever_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_lever_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_tube_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_tube_or_not_merge['dodson'])[0],1)/fps - session_start_time 
        look_at_otherlever_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_otherlever_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_otherface_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_otherface_or_not_merge['dodson'])[0],1)/fps - session_start_time

        
        # find time point of behavioral events
        # output_time_points_socialgaze ,output_time_points_levertube = bhv_events_timepoint_singlecam(bhv_data,look_at_other_or_not_merge,look_at_lever_or_not_merge,look_at_tube_or_not_merge)
        output_time_points_socialgaze ,output_time_points_levertube = bhv_events_timepoint_singlecam(bhv_data,look_at_otherface_or_not_merge,look_at_lever_or_not_merge,look_at_tube_or_not_merge)
        time_point_pull1 = output_time_points_socialgaze['time_point_pull1']
        time_point_pull2 = output_time_points_socialgaze['time_point_pull2']
        oneway_gaze1 = output_time_points_socialgaze['oneway_gaze1']
        oneway_gaze2 = output_time_points_socialgaze['oneway_gaze2']
        mutual_gaze1 = output_time_points_socialgaze['mutual_gaze1']
        mutual_gaze2 = output_time_points_socialgaze['mutual_gaze2']
        time_point_lever1 = output_time_points_levertube['time_point_lookatlever1']
        time_point_lever2 = output_time_points_levertube['time_point_lookatlever2']
        time_point_tube1 = output_time_points_levertube['time_point_lookattube1']
        time_point_tube2 = output_time_points_levertube['time_point_lookattube2']
            
        output_time_points_otherlever = bhv_events_timepoint_singlecam_otherlever(bhv_data, look_at_otherlever_or_not_merge)
        time_point_otherlever1 = output_time_points_otherlever['time_point_lookatotherlever1']
        time_point_otherlever2 = output_time_points_otherlever['time_point_lookatotherlever2']
        
        
            
        #
        if np.isin(animal1,animal1_fixedorder):
            owgaze1_num_all_dates[idate] = np.shape(oneway_gaze1)[0]#/(min_length/fps)
            owgaze2_num_all_dates[idate] = np.shape(oneway_gaze2)[0]#/(min_length/fps)
            mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze1)[0]#/(min_length/fps)
            mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze2)[0]#/(min_length/fps)
            gazelever1_num_all_dates[idate] = np.shape(time_point_lever1)[0]
            gazelever2_num_all_dates[idate] = np.shape(time_point_lever2)[0]
            gazetube1_num_all_dates[idate] = np.shape(time_point_tube1)[0]
            gazetube2_num_all_dates[idate] = np.shape(time_point_tube2)[0]
            
            gazeotherlever1_num_all_dates[idate] = np.shape(time_point_otherlever1)[0]
            gazeotherlever2_num_all_dates[idate] = np.shape(time_point_otherlever2)[0]
        else:
            owgaze1_num_all_dates[idate] = np.shape(oneway_gaze2)[0]#/(min_length/fps)
            owgaze2_num_all_dates[idate] = np.shape(oneway_gaze1)[0]#/(min_length/fps)
            mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze2)[0]#/(min_length/fps)
            mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze1)[0]#/(min_length/fps)
            gazelever1_num_all_dates[idate] = np.shape(time_point_lever2)[0]
            gazelever2_num_all_dates[idate] = np.shape(time_point_lever1)[0]
            gazetube1_num_all_dates[idate] = np.shape(time_point_tube2)[0]
            gazetube2_num_all_dates[idate] = np.shape(time_point_tube1)[0]
            
            gazeotherlever1_num_all_dates[idate] = np.shape(time_point_otherlever2)[0]
            gazeotherlever2_num_all_dates[idate] = np.shape(time_point_otherlever1)[0]

        
        # plot and analyze the gaze distribution along the phase of continous behavioral variables
        if 1:
            fig_savepath = data_saved_folder+'/	bhv_events_continuous_variables_singlecam_wholebody/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+cameraID+'/'+date_tgt+'/'   
            if not os.path.exists(fig_savepath):
                os.makedirs(fig_savepath)
            fig_savepath = fig_savepath + date_tgt

            doActvePeri = 1
            doGazeStart = 1
            savefig = 1
            #
            gazeDist_phaseof_contbhvvar_summary = plot_gaze_along_phase_of_continuous_bhv_var_singlecam(fig_savepath, savefig, animal1, animal2, session_start_time, 
                                                                            time_point_pull1, time_point_pull2, oneway_gaze1, oneway_gaze2, 
                                                                            mutual_gaze1, mutual_gaze2, animalnames_videotrack, output_look_ornot, 
                                                                            output_allvectors, output_allangles, output_key_locations, doActvePeri, doGazeStart)
            #
            gazeDist_phaseof_contbhvvar_all_dates[date_tgt] = gazeDist_phaseof_contbhvvar_summary

        
        # analyze the events interval, especially for the pull to other and other to pull interval
        # could be used for define time bin for DBN
        if np.isin(animal1,animal1_fixedorder):
            _,_,_,pullTOother_itv, otherTOpull_itv = bhv_events_interval(totalsess_time, session_start_time, time_point_pull1, time_point_pull2, 
                                                                         oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
            #
            pull_other_pool_itv = np.concatenate((pullTOother_itv,otherTOpull_itv))
            bhv_intv_all_dates[date_tgt] = {'pull_to_other':pullTOother_itv,'other_to_pull':otherTOpull_itv,
                            'pull_other_pooled': pull_other_pool_itv}
            
            all_pull_edges_intervals = bhv_events_interval_certainEdges(totalsess_time, session_start_time, time_point_pull1, time_point_pull2, 
                                                                        oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
            pull_edges_intv_all_dates[date_tgt] = all_pull_edges_intervals
        else:
            _,_,_,pullTOother_itv, otherTOpull_itv = bhv_events_interval(totalsess_time, session_start_time, time_point_pull2, time_point_pull1, 
                                                                         oneway_gaze2, oneway_gaze1, mutual_gaze2, mutual_gaze1)
            #
            pull_other_pool_itv = np.concatenate((pullTOother_itv,otherTOpull_itv))
            bhv_intv_all_dates[date_tgt] = {'pull_to_other':pullTOother_itv,'other_to_pull':otherTOpull_itv,
                            'pull_other_pooled': pull_other_pool_itv}
            
            all_pull_edges_intervals = bhv_events_interval_certainEdges(totalsess_time, session_start_time, time_point_pull2, time_point_pull1, 
                                                                        oneway_gaze2, oneway_gaze1, mutual_gaze2, mutual_gaze1)
            pull_edges_intv_all_dates[date_tgt] = all_pull_edges_intervals
   
        
        

    # save data
    if 1:
        
        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
        if not os.path.exists(data_saved_subfolder):
            os.makedirs(data_saved_subfolder)
                
        # with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
        #     pickle.dump(DBN_input_data_alltypes, f)

        with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze1_num_all_dates, f)
        with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze2_num_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze1_num_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze2_num_all_dates, f)
        with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull1_num_all_dates, f)
        with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull2_num_all_dates, f)
            
        with open(data_saved_subfolder+'/gazelever1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazelever1_num_all_dates, f)
        with open(data_saved_subfolder+'/gazelever2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazelever2_num_all_dates, f)
        with open(data_saved_subfolder+'/gazetube1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazetube1_num_all_dates, f)
        with open(data_saved_subfolder+'/gazetube2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazetube2_num_all_dates, f)
         
        with open(data_saved_subfolder+'/session_recordtime_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(session_recordtime_all_dates, f)
        
        with open(data_saved_subfolder+'/gazeotherlever1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazeotherlever1_num_all_dates, f)
        with open(data_saved_subfolder+'/gazeotherlever2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazeotherlever2_num_all_dates, f)
        

        with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(tasktypes_all_dates, f)
        with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(coopthres_all_dates, f)
        with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(succ_rate_all_dates, f)
        with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(interpullintv_all_dates, f)
        with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(trialnum_all_dates, f)
        with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(bhv_intv_all_dates, f)
        with open(data_saved_subfolder+'/pull_edges_intv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull_edges_intv_all_dates, f)
    
        with open(data_saved_subfolder+'/gazeDist_phaseof_contbhvvar_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(gazeDist_phaseof_contbhvvar_all_dates, f)
    

In [ ]:
if 0:
    minframe = 5000
    maxframe = 7000
    
    gaze_thresold = 0.25
    
    # get the discrete behavioral events
    # aligned to the start of the video recording
    time_point_pull1_frames = (np.array(time_point_pull1)+session_start_time)*fps 
    time_point_pull2_frames = (np.array(time_point_pull2)+session_start_time)*fps 

    # define gazes are all gaze entry
    #
    # oneway_gaze1_frames = (np.sort(np.concatenate((oneway_gaze1,mutual_gaze1)))+session_start_time)*fps 
    # oneway_gaze2_frames = (np.sort(np.concatenate((oneway_gaze2,mutual_gaze2)))+session_start_time)*fps 

    # define gaze as the flash gaze and gaze start
    # get the gaze start and stop
    #animal1_gaze = np.concatenate([oneway_gaze1, mutual_gaze1])
    animal1_gaze = np.sort(np.concatenate((oneway_gaze1,mutual_gaze1)))
    animal1_gaze = np.sort(np.unique(animal1_gaze))
    animal1_gaze_stop = animal1_gaze[np.concatenate(((animal1_gaze[1:]-animal1_gaze[0:-1]>gaze_thresold)*1,[1]))==1]
    animal1_gaze_start = np.concatenate(([animal1_gaze[0]],animal1_gaze[np.where(animal1_gaze[1:]-animal1_gaze[0:-1]>gaze_thresold)[0]+1]))
    animal1_gaze_flash = np.intersect1d(animal1_gaze_start, animal1_gaze_stop)
    animal1_gaze_start = animal1_gaze_start[~np.isin(animal1_gaze_start,animal1_gaze_flash)]
    animal1_gaze_stop = animal1_gaze_stop[~np.isin(animal1_gaze_stop,animal1_gaze_flash)]
    #
    #animal2_gaze = np.concatenate([oneway_gaze2, mutual_gaze2])
    animal2_gaze = np.sort(np.concatenate((oneway_gaze2,mutual_gaze2)))
    animal2_gaze = np.sort(np.unique(animal2_gaze))
    animal2_gaze_stop = animal2_gaze[np.concatenate(((animal2_gaze[1:]-animal2_gaze[0:-1]>gaze_thresold)*1,[1]))==1]
    animal2_gaze_start = np.concatenate(([animal2_gaze[0]],animal2_gaze[np.where(animal2_gaze[1:]-animal2_gaze[0:-1]>gaze_thresold)[0]+1]))
    animal2_gaze_flash = np.intersect1d(animal2_gaze_start, animal2_gaze_stop)
    animal2_gaze_start = animal2_gaze_start[~np.isin(animal2_gaze_start,animal2_gaze_flash)]
    animal2_gaze_stop = animal2_gaze_stop[~np.isin(animal2_gaze_stop,animal2_gaze_flash)] 
    #
    oneway_gaze1_frames = (np.sort(np.concatenate((animal1_gaze_start,animal1_gaze_flash)))+session_start_time)*fps 
    oneway_gaze2_frames = (np.sort(np.concatenate((animal2_gaze_start,animal2_gaze_flash)))+session_start_time)*fps 

    a = output_key_locations['lever_loc_all_merge']['dodson'].transpose()
    b = output_key_locations['meaneye_loc_all_merge']['dodson'].transpose()
    a_min_b = a - b
    otherani_otherlever_dist = np.sqrt(np.einsum('ij,ij->j', a_min_b, a_min_b))
    otherani_otherlever_dist = scipy.ndimage.gaussian_filter1d(otherani_otherlever_dist,3)

    otherani_otherlever_dist = (otherani_otherlever_dist-np.nanmin(otherani_otherlever_dist))/(np.nanmax(otherani_otherlever_dist)-np.nanmin(otherani_otherlever_dist))

    oneway_gaze1_frames_forplot = oneway_gaze1_frames[(oneway_gaze1_frames<maxframe)&(oneway_gaze1_frames>minframe)]
    oneway_gaze2_frames_forplot = oneway_gaze2_frames[(oneway_gaze2_frames<maxframe)&(oneway_gaze2_frames>minframe)]

    pull1_frames_forplot = time_point_pull1_frames[(time_point_pull1_frames<maxframe)&(time_point_pull1_frames>minframe)]
    pull2_frames_forplot = time_point_pull2_frames[(time_point_pull2_frames<maxframe)&(time_point_pull2_frames>minframe)]

    fig, ax1 = plt.subplots(figsize=(10, 5))

    for igaze in oneway_gaze2_frames_forplot:
        ax1.plot([igaze,igaze],[0,1],color='#7c7c7c')

    for ipull in pull1_frames_forplot:
        ax1.plot([ipull,ipull],[0,1],color='r')

    for ipull in pull2_frames_forplot:
        ax1.plot([ipull,ipull],[0,1],color='b')

    ax1.plot(np.arange(minframe,maxframe,1),otherani_otherlever_dist[minframe:maxframe])



### prepare the input data for DBN

In [ ]:
# define DBN related summarizing variables
DBN_input_data_alltypes = dict.fromkeys(dates_list, [])

doBhvitv_timebin = 0 # 1: if use the mean bhv event interval for time bin

prepare_input_data = 0

# DBN resolutions (make sure they are the same as in the later part of the code)
# totalsess_time = 600 # total session time in s
# temp_resolus = [0.5,1,1.5,2] # temporal resolution in the DBN model, eg: 0.5 means 500ms
temp_resolus = [1] # temporal resolution in the DBN model, eg: 0.5 means 500ms
ntemp_reses = np.shape(temp_resolus)[0]

mergetempRos = 0

# # train the dynamic bayesian network - Alec's model 
#   prepare the multi-session table; one time lag; multi time steps (temporal resolution) as separate files

# prepare the DBN input data
if prepare_input_data:
    
    for idate in np.arange(0,ndates,1):
        date_tgt = dates_list[idate]
        session_start_time = session_start_times[idate]
        session_recordtime_all = session_recordtime_all_dates[idate]
        
        totalsess_time = np.ceil(session_recordtime_all-session_start_time)
        

        # load behavioral results
        try:
            try:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_from_task_code/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path +date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
            except:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_from_task_code/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
        except:    
            try:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_forceManipulation_task/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path +date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
            except:
                bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_forceManipulation_task/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
                trial_record_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_TrialRecord_" + "*.json")
                bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_bhv_data_" + "*.json")
                session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_session_info_" + "*.json")
                #
                trial_record = pd.read_json(trial_record_json[0])
                bhv_data = pd.read_json(bhv_data_json[0])
                session_info = pd.read_json(session_info_json[0])
            
        # get animal info
        animal1 = session_info['lever1_animal'][0].lower()
        animal2 = session_info['lever2_animal'][0].lower()
        
        # clean up the trial_record
        warnings.filterwarnings('ignore')
        trial_record_clean = pd.DataFrame(columns=trial_record.columns)
        for itrial in np.arange(0,np.max(trial_record['trial_number']),1):
            # trial_record_clean.loc[itrial] = trial_record[trial_record['trial_number']==itrial+1].iloc[[0]]
            trial_record_clean = trial_record_clean.append(trial_record[trial_record['trial_number']==itrial+1].iloc[[0]])
        trial_record_clean = trial_record_clean.reset_index(drop = True)

        # change bhv_data time to the absolute time
        time_points_new = pd.DataFrame(np.zeros(np.shape(bhv_data)[0]),columns=["time_points_new"])
        for itrial in np.arange(0,np.max(trial_record_clean['trial_number']),1):
            ind = bhv_data["trial_number"]==itrial+1
            new_time_itrial = bhv_data[ind]["time_points"] + trial_record_clean["trial_starttime"].iloc[itrial]
            time_points_new["time_points_new"][ind] = new_time_itrial
        bhv_data["time_points"] = time_points_new["time_points_new"]
        bhv_data = bhv_data[bhv_data["time_points"] != 0]
            
        # get task type and cooperation threshold
        try:
            coop_thres = session_info["pulltime_thres"][0]
            tasktype = session_info["task_type"][0]
        except:
            coop_thres = 0
            tasktype = 1

        # load behavioral event results
        print('load social gaze with '+cameraID+' only of '+date_tgt)
        with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_look_ornot.pkl', 'rb') as f:
            output_look_ornot = pickle.load(f)
        with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allvectors.pkl', 'rb') as f:
            output_allvectors = pickle.load(f)
        with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allangles.pkl', 'rb') as f:
            output_allangles = pickle.load(f)  
        #
        look_at_other_or_not_merge = output_look_ornot['look_at_other_or_not_merge']
        look_at_tube_or_not_merge = output_look_ornot['look_at_tube_or_not_merge']
        look_at_lever_or_not_merge = output_look_ornot['look_at_lever_or_not_merge']
        look_at_otherlever_or_not_merge = output_look_ornot['look_at_otherlever_or_not_merge']
        look_at_otherface_or_not_merge = output_look_ornot['look_at_otherface_or_not_merge']
        
        # change the unit to second
        session_start_time = session_start_times[idate]
        look_at_other_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_other_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_lever_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_lever_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_tube_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_tube_or_not_merge['dodson'])[0],1)/fps - session_start_time 
        look_at_otherlever_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_otherlever_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_otherface_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_otherface_or_not_merge['dodson'])[0],1)/fps - session_start_time
        
        # find time point of behavioral events
        # output_time_points_socialgaze ,output_time_points_levertube = bhv_events_timepoint_singlecam(bhv_data,look_at_other_or_not_merge,look_at_lever_or_not_merge,look_at_tube_or_not_merge)
        output_time_points_socialgaze ,output_time_points_levertube = bhv_events_timepoint_singlecam(bhv_data,look_at_otherface_or_not_merge,look_at_lever_or_not_merge,look_at_tube_or_not_merge)
        time_point_pull1 = output_time_points_socialgaze['time_point_pull1']
        time_point_pull2 = output_time_points_socialgaze['time_point_pull2']
        oneway_gaze1 = output_time_points_socialgaze['oneway_gaze1']
        oneway_gaze2 = output_time_points_socialgaze['oneway_gaze2']
        mutual_gaze1 = output_time_points_socialgaze['mutual_gaze1']
        mutual_gaze2 = output_time_points_socialgaze['mutual_gaze2']   
        time_point_lever1 = output_time_points_levertube['time_point_lookatlever1']
        time_point_lever2 = output_time_points_levertube['time_point_lookatlever2']
        time_point_tube1 = output_time_points_levertube['time_point_lookattube1']
        time_point_tube2 = output_time_points_levertube['time_point_lookattube2']
            
        output_time_points_otherlever = bhv_events_timepoint_singlecam_otherlever(bhv_data, look_at_otherlever_or_not_merge)
        time_point_otherlever1 = output_time_points_otherlever['time_point_lookatotherlever1']
        time_point_otherlever2 = output_time_points_otherlever['time_point_lookatotherlever2']

        
        if mergetempRos:
            temp_resolus = [0.5,1,1.5,2] # temporal resolution in the DBN model, eg: 0.5 means 500ms
            # use bhv event to decide temporal resolution
            #
            #low_lim,up_lim,_ = bhv_events_interval(totalsess_time, session_start_time, time_point_pull1, time_point_pull2, oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
            #temp_resolus = temp_resolus = np.arange(low_lim,up_lim,0.1)
        #
        if doBhvitv_timebin:
            pull_other_intv_ii = pd.Series(bhv_intv_all_dates[date_tgt]['pull_other_pooled'])
            # remove the interval that is too large
            pull_other_intv_ii[pull_other_intv_ii>(np.nanmean(pull_other_intv_ii)+2*np.nanstd(pull_other_intv_ii))]= np.nan    
            # pull_other_intv_ii[pull_other_intv_ii>10]= np.nan
            temp_resolus = [np.nanmean(pull_other_intv_ii)]          
        #
        ntemp_reses = np.shape(temp_resolus)[0]           

        
        # try different temporal resolutions
        for temp_resolu in temp_resolus:
            bhv_df = []

            if np.isin(animal1,animal1_fixedorder):
                bhv_df_itr,_,_ = train_DBN_multiLag_create_df_only(totalsess_time, session_start_time, temp_resolu, 
                                                                   time_point_pull1, time_point_pull2, 
                                                                   oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2,
                                                                   time_point_otherlever1, time_point_otherlever2)
            else:
                bhv_df_itr,_,_ = train_DBN_multiLag_create_df_only(totalsess_time, session_start_time, temp_resolu, 
                                                                   time_point_pull2, time_point_pull1, 
                                                                   oneway_gaze2, oneway_gaze1, mutual_gaze2, mutual_gaze1,
                                                                   time_point_otherlever2, time_point_otherlever1)     

            if len(bhv_df)==0:
                bhv_df = bhv_df_itr
            else:
                bhv_df = pd.concat([bhv_df,bhv_df_itr])                   
                bhv_df = bhv_df.reset_index(drop=True)        

            DBN_input_data_alltypes[date_tgt] = bhv_df
            
    # save data
    if 1:
        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody_allsessions'+savefile_sufix+'_3lags/'+cameraID+'/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
        if not os.path.exists(data_saved_subfolder):
            os.makedirs(data_saved_subfolder)
        if not mergetempRos:
            if doBhvitv_timebin:
                with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'_'+str(temp_resolu)+'bhvItvTempReSo.pkl', 'wb') as f:
                    pickle.dump(DBN_input_data_alltypes, f)
            else:
                with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'_'+str(temp_resolu)+'sReSo.pkl', 'wb') as f:
                    pickle.dump(DBN_input_data_alltypes, f)
        else:
            with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'_mergeTempsReSo.pkl', 'wb') as f:
                pickle.dump(DBN_input_data_alltypes, f)     

In [ ]:
DBN_input_data_alltypes.keys()

#### redefine the tasktype and cooperation threshold to merge them together

In [ ]:
# 100: self; 3: 3s coop; 2: 2s coop; 1.5: 1.5s coop; 1: 1s coop; -1: no-vision; 3.5:animal1 autolever, 4:animal2_autolever

tasktypes_all_dates[tasktypes_all_dates==5] = -1 # change the task type code for no-vision
coopthres_forsort = (tasktypes_all_dates-1)*coopthres_all_dates/2
coopthres_forsort[coopthres_forsort==0] = 0 # get the cooperation threshold for sorting

### plot behavioral events interval to get a sense about time bin
#### only focus on pull_to_other_bhv_interval and other_bhv_to_pull_interval

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
#
# sort the data based on task type and dates
sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [True, True])
dates_list_sorted = np.array(dates_list)[sorting_df.index]
ndates_sorted = np.shape(dates_list_sorted)[0]

pull_other_intv_forplots = {}
pull_other_intv_mean = np.zeros((1,ndates_sorted))[0]
pull_other_intv_ii = []
for ii in np.arange(0,ndates_sorted,1):
    pull_other_intv_ii = pd.Series(bhv_intv_all_dates[dates_list_sorted[ii]]['pull_other_pooled'])
    # remove the interval that is too large
    pull_other_intv_ii[pull_other_intv_ii>(np.nanmean(pull_other_intv_ii)+2*np.nanstd(pull_other_intv_ii))]= np.nan    
    # pull_other_intv_ii[pull_other_intv_ii>10]= np.nan
    pull_other_intv_forplots[ii] = pull_other_intv_ii
    pull_other_intv_mean[ii] = np.nanmean(pull_other_intv_ii)
    
    
#
pull_other_intv_forplots = pd.DataFrame(pull_other_intv_forplots)

#
# plot
pull_other_intv_forplots.plot(kind = 'box',ax=ax1, positions=np.arange(0,ndates_sorted,1))
# plt.boxplot(pull_other_intv_forplots)
plt.plot(np.arange(0,ndates_sorted,1),pull_other_intv_mean,'r*',markersize=10)
#
ax1.set_ylabel("bhv event interval(around pulls)",fontsize=13)
ax1.set_ylim([-2,16])
#
plt.xticks(np.arange(0,ndates_sorted,1),dates_list_sorted, rotation=90,fontsize=10)
plt.yticks(fontsize=10)
#
tasktypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']
taskswitches = np.where(np.array(sorting_df['coopthres'])[1:]-np.array(sorting_df['coopthres'])[:-1]!=0)[0]+0.5
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.plot([taskswitch,taskswitch],[-2,15],'k--')
taskswitches = np.concatenate(([0],taskswitches))
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.text(taskswitch+0.25,-1,tasktypes[itaskswitch],fontsize=10)
ax1.text(taskswitch-5,15,'mean Inteval = '+str(np.nanmean(pull_other_intv_forplots)),fontsize=10)

print(pull_other_intv_mean)
print(np.nanmean(pull_other_intv_forplots))

savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"bhvInterval_hist_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')


### plot some other basis behavioral measures
#### successful rate

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
#
# sort the data based on task type and dates
sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [True, True])
dates_list_sorted = np.array(dates_list)[sorting_df.index]
ndates_sorted = np.shape(dates_list_sorted)[0]


ax1.plot(np.arange(0,ndates_sorted,1),succ_rate_all_dates[sorting_df.index],'o',markersize=10)
#
ax1.set_ylabel("successful rate",fontsize=13)
ax1.set_ylim([-0.1,1.1])
ax1.set_xlim([-0.5,ndates_sorted-0.5])
#
plt.xticks(np.arange(0,ndates_sorted,1),dates_list_sorted, rotation=90,fontsize=10)
plt.yticks(fontsize=10)
#
tasktypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']
taskswitches = np.where(np.array(sorting_df['coopthres'])[1:]-np.array(sorting_df['coopthres'])[:-1]!=0)[0]+0.5
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.plot([taskswitch,taskswitch],[-0.1,1.1],'k--')
taskswitches = np.concatenate(([0],taskswitches))
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.text(taskswitch+0.25,-0.05,tasktypes[itaskswitch],fontsize=10)
    
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"successfulrate_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')



#### animal pull numbers

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
#
# sort the data based on task type and dates
sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [True, True])
dates_list_sorted = np.array(dates_list)[sorting_df.index]
ndates_sorted = np.shape(dates_list_sorted)[0]

pullmean_num_all_dates = (pull1_num_all_dates+pull2_num_all_dates)/2
ax1.plot(np.arange(0,ndates_sorted,1),
         pull1_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'bv',markersize=5,label='animal1 pull #')
ax1.plot(np.arange(0,ndates_sorted,1),
         pull2_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'rv',markersize=5,label='animal2 pull #')
ax1.plot(np.arange(0,ndates_sorted,1),
         pullmean_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'kv',markersize=8,label='mean pull #')
ax1.legend()


#
ax1.set_ylabel("pull numbers per second",fontsize=13)
ax1.set_ylim([-0.1,0.4])
ax1.set_xlim([-0.5,ndates_sorted-0.5])

#
plt.xticks(np.arange(0,ndates_sorted,1),dates_list_sorted, rotation=90,fontsize=10)
plt.yticks(fontsize=10)
#
tasktypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']
taskswitches = np.where(np.array(sorting_df['coopthres'])[1:]-np.array(sorting_df['coopthres'])[:-1]!=0)[0]+0.5
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.plot([taskswitch,taskswitch],[-20,540],'k--')
taskswitches = np.concatenate(([0],taskswitches))
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.text(taskswitch+0.25,-0.05,tasktypes[itaskswitch],fontsize=10)
    
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"pullnumbers_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')



#### gaze number

In [ ]:

gaze1_num_all_dates = owgaze1_num_all_dates + mtgaze1_num_all_dates
gaze2_num_all_dates = owgaze2_num_all_dates + mtgaze2_num_all_dates
gazemean_num_all_dates = (gaze1_num_all_dates+gaze2_num_all_dates)/2

print(np.nanmax(gaze1_num_all_dates))
print(np.nanmax(gaze2_num_all_dates))

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
#
# sort the data based on task type and dates
sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [True, True])
dates_list_sorted = np.array(dates_list)[sorting_df.index]
ndates_sorted = np.shape(dates_list_sorted)[0]



ax1.plot(np.arange(0,ndates_sorted,1),
         gaze1_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'b^',markersize=5,label='animal1 gaze #')
ax1.plot(np.arange(0,ndates_sorted,1),
         gaze2_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'r^',markersize=5,label='animal2 gaze #')
ax1.plot(np.arange(0,ndates_sorted,1),
         gazemean_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'k^',markersize=8,label='mean gaze #')
ax1.legend()


#
ax1.set_ylabel("social gaze number per second",fontsize=13)
ax1.set_ylim([-0.1,3])
ax1.set_xlim([-0.5,ndates_sorted-0.5])

#
plt.xticks(np.arange(0,ndates_sorted,1),dates_list_sorted, rotation=90,fontsize=10)
plt.yticks(fontsize=10)
#
tasktypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']
taskswitches = np.where(np.array(sorting_df['coopthres'])[1:]-np.array(sorting_df['coopthres'])[:-1]!=0)[0]+0.5
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.plot([taskswitch,taskswitch],[-20,4500],'k--')
taskswitches = np.concatenate(([0],taskswitches))
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.text(taskswitch+0.25,-0.05,tasktypes[itaskswitch],fontsize=10)
    
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"gazenumbers_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')


In [ ]:
gazeotherlever_mean_all_dates = (gazeotherlever1_num_all_dates+gazeotherlever2_num_all_dates)/2

fig, ax1 = plt.subplots(figsize=(10, 5))
#
# sort the data based on task type and dates
sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [True, True])
dates_list_sorted = np.array(dates_list)[sorting_df.index]
ndates_sorted = np.shape(dates_list_sorted)[0]



ax1.plot(np.arange(0,ndates_sorted,1),
         gazeotherlever1_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'b^',markersize=5,label='animal1 gaze #')
ax1.plot(np.arange(0,ndates_sorted,1),
         gazeotherlever2_num_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'r^',markersize=5,label='animal2 gaze #')
ax1.plot(np.arange(0,ndates_sorted,1),
         gazeotherlever_mean_all_dates[sorting_df.index]/session_recordtime_all_dates[sorting_df.index],
         'k^',markersize=8,label='mean gaze #')
ax1.legend()


#
ax1.set_ylabel("gaze other lever number per second",fontsize=13)
ax1.set_ylim([-0.1,3])
ax1.set_xlim([-0.5,ndates_sorted-0.5])

#
plt.xticks(np.arange(0,ndates_sorted,1),dates_list_sorted, rotation=90,fontsize=10)
plt.yticks(fontsize=10)
#
tasktypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']
taskswitches = np.where(np.array(sorting_df['coopthres'])[1:]-np.array(sorting_df['coopthres'])[:-1]!=0)[0]+0.5
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.plot([taskswitch,taskswitch],[-20,4500],'k--')
taskswitches = np.concatenate(([0],taskswitches))
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.text(taskswitch+0.25,-0.05,tasktypes[itaskswitch],fontsize=10)
    
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"gazeotherlevernumbers_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')


In [ ]:
gaze_numbers = (owgaze1_num_all_dates+owgaze2_num_all_dates+mtgaze1_num_all_dates+mtgaze2_num_all_dates)/session_recordtime_all_dates
gaze_pull_ratios = (owgaze1_num_all_dates+owgaze2_num_all_dates+mtgaze1_num_all_dates+mtgaze2_num_all_dates)/(pull1_num_all_dates+pull2_num_all_dates)

gazeotherlever_numbers = (gazeotherlever1_num_all_dates+gazeotherlever2_num_all_dates)/session_recordtime_all_dates

fig, ax1 = plt.subplots(figsize=(10, 5))

grouptypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']

gaze_numbers_groups = [np.transpose(gaze_numbers[np.transpose(coopthres_forsort==0)[0]])[0],
                       np.transpose(gaze_numbers[np.transpose(coopthres_forsort==1)[0]])[0],
                       np.transpose(gaze_numbers[np.transpose(coopthres_forsort==3.5)[0]])[0],
                       np.transpose(gaze_numbers[np.transpose(coopthres_forsort==4)[0]])[0],]

gazeotherlever_numbers_groups = [np.transpose(gazeotherlever_numbers[np.transpose(coopthres_forsort==0)[0]])[0],
                                 np.transpose(gazeotherlever_numbers[np.transpose(coopthres_forsort==1)[0]])[0],
                                 np.transpose(gazeotherlever_numbers[np.transpose(coopthres_forsort==3.5)[0]])[0],
                                 np.transpose(gazeotherlever_numbers[np.transpose(coopthres_forsort==4)[0]])[0],]

gaze_numbers_plot = plt.boxplot(gaze_numbers_groups)
plt.boxplot(gazeotherlever_numbers_groups)

plt.xticks(np.arange(1, len(grouptypes)+1, 1), grouptypes, fontsize = 12);
ax1.set_ylim([-0.1,4])
ax1.set_ylabel("average social gaze number per second",fontsize=13)

savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"averaged_gazenumbers_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')


### plot the gaze distribution along the continuous variables
#### only focus on "otherani_otherlever_dist", plot the ratio of number of 'decrasing' phase vs 'increasing' phase

In [ ]:
# fig, axs = plt.subplots(2,1)
# fig.set_figheight(10)
# fig.set_figwidth(10*2)

fig, ax1 = plt.subplots(figsize=(10, 5))

type_tgt = 'otherani_otherlever_dist'

# sort the data based on task type and dates
sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [False, True])
dates_list_sorted = np.array(dates_list)[sorting_df.index]
ndates_sorted = np.shape(dates_list_sorted)[0]

gazeDist_ratios_forplot_animal1 = np.ones(np.shape(dates_list_sorted))*np.nan
gazeDist_ratios_forplot_animal2 = np.ones(np.shape(dates_list_sorted))*np.nan

for idate in np.arange(0,ndates_sorted,1):
    
    date_i = dates_list_sorted[idate]
    
    # for animal 1
    gazeDist_tgt = gazeDist_phaseof_contbhvvar_all_dates[date_i][animal1_fixedorder[0]][type_tgt]
    gateDist_ratio_tgt = np.sum(gazeDist_tgt['phase']=='decreasing')/np.sum(gazeDist_tgt['phase']=='increasing')
    gazeDist_ratios_forplot_animal1[idate] = gateDist_ratio_tgt
    
    # for animal 2
    gazeDist_tgt = gazeDist_phaseof_contbhvvar_all_dates[date_i][animal2_fixedorder[0]][type_tgt]
    gateDist_ratio_tgt = np.sum(gazeDist_tgt['phase']=='decreasing')/np.sum(gazeDist_tgt['phase']=='increasing')
    gazeDist_ratios_forplot_animal2[idate] = gateDist_ratio_tgt
    

ax1.plot(np.arange(0,ndates_sorted,1),gazeDist_ratios_forplot_animal1,'b^',markersize=5,label=animal1_fixedorder[0])
ax1.plot(np.arange(0,ndates_sorted,1),gazeDist_ratios_forplot_animal2,'r^',markersize=5,label=animal2_fixedorder[0])
ax1.plot([-0.5,ndates_sorted-0.5],[1,1],'k--')
ax1.legend()

#
ax1.set_ylabel('ratio',fontsize=13)
ax1.set_title("social gaze on the decreasing phase / inecreasing phase of "+type_tgt)
# ax1.set_ylim([-20,1500])
ax1.set_ylim([0,2])
ax1.set_xlim([-0.5,ndates_sorted-0.5])

#
plt.xticks(np.arange(0,ndates_sorted,1),dates_list_sorted, rotation=90,fontsize=10)
plt.yticks(fontsize=10)
#
tasktypes = ['self','coop(1s)','animal1_autolever','animal2_autolever']
taskswitches = np.where(np.array(sorting_df['coopthres'])[1:]-np.array(sorting_df['coopthres'])[:-1]!=0)[0]+0.5
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    # ax1.plot([taskswitch,taskswitch],[-20,1500],'k--')
    ax1.plot([taskswitch,taskswitch],[0,2],'k--')
taskswitches = np.concatenate(([0],taskswitches))
for itaskswitch in np.arange(0,np.shape(taskswitches)[0],1):
    taskswitch = taskswitches[itaskswitch]
    ax1.text(taskswitch+0.25,0.5,tasktypes[itaskswitch],fontsize=10)
    
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"gazenumberratio_decreasingphaseVSincreasingphase"+type_tgt+"_"+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pdf')

## plot that includes all the animals - final summarizing plot
#### plot the gaze numbers for all individuals 

In [ ]:
# animal1_fixedorders = ['eddie','dodson','dannon','ginger']
# animal2_fixedorders = ['sparkle','scorch','kanga_1','kanga_2']
animal1_fixedorders = ['eddie','dodson','ginger']
animal2_fixedorders = ['sparkle','scorch','kanga_2']
nanimalpairs = np.shape(animal1_fixedorders)[0]

# grouptypes = ['self reward','1s threshold','animal1 autolever','animal2_autolever']
grouptypes = ['self reward','1s threshold','self autolever','other autolever']
coopthres_IDs = [0, 1, 3.5, 4]
ngrouptypes = np.shape(grouptypes)[0]

gazenum_foreachgroup_foreachAni = dict.fromkeys(grouptypes,[])
gazenum_foreachgroup_all = dict.fromkeys(grouptypes,[])
#
gazeothernum_foreachgroup_foreachAni = dict.fromkeys(grouptypes,[])
gazeothernum_foreachgroup_all = dict.fromkeys(grouptypes,[])

#
# malenames = ['eddie','dodson','dannon',]
# femalenames = ['sparkle','scorch','kanga_1','kanga_2','ginger',]
malenames = ['eddie','dodson',]
femalenames = ['sparkle','scorch','kanga_2','ginger',]
gazenum_foreachgroup_male = dict.fromkeys(grouptypes,[])
gazenum_foreachgroup_female = dict.fromkeys(grouptypes,[])
#
gazeothernum_foreachgroup_male = dict.fromkeys(grouptypes,[])
gazeothernum_foreachgroup_female = dict.fromkeys(grouptypes,[])

#
# subnames = ['eddie','dodson','dannon','ginger',]
# domnames = ['sparkle','scorch','kanga_1','kanga_2',]
subnames = ['eddie','dodson','ginger',]
domnames = ['sparkle','scorch','kanga_2',]
gazenum_foreachgroup_sub = dict.fromkeys(grouptypes,[])
gazenum_foreachgroup_dom = dict.fromkeys(grouptypes,[])
#
gazeothernum_foreachgroup_sub = dict.fromkeys(grouptypes,[])
gazeothernum_foreachgroup_dom = dict.fromkeys(grouptypes,[])

#
for igrouptype in np.arange(0,ngrouptypes,1):
    
    grouptype = grouptypes[igrouptype]
    coopthres_ID = coopthres_IDs[igrouptype]
    
    gazenum_foreachgroup_foreachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])
    gazeothernum_foreachgroup_foreachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])
    

    #
    for ianimalpair in np.arange(0,nanimalpairs,1):
        animal1 = animal1_fixedorders[ianimalpair]
        animal2 = animal2_fixedorders[ianimalpair]
        
        if (animal2 == 'kanga_1') | (animal2 == 'kanga_2'):
            animal2_filename = 'kanga'
        else:
            animal2_filename = animal2

        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1+animal2_filename+'/'

        with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            owgaze1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            owgaze2_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            mtgaze1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            mtgaze2_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            pull1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            pull2_num_all_dates = pickle.load(f)

        with open(data_saved_subfolder+'/gazelever1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazelever1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazelever2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazelever2_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazetube1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazetube1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazetube2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazetube2_num_all_dates = pickle.load(f)

        with open(data_saved_subfolder+'/gazeotherlever1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazeotherlever1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazeotherlever2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazeotherlever2_num_all_dates = pickle.load(f)

        with open(data_saved_subfolder+'/session_recordtime_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            session_recordtime_all_dates = pickle.load(f)
        
        with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            tasktypes_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            coopthres_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            succ_rate_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            interpullintv_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            trialnum_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            bhv_intv_all_dates = pickle.load(f)

        # combine owgaze and mtgaze
        gaze1_num_all_dates = (owgaze1_num_all_dates + mtgaze1_num_all_dates)/session_recordtime_all_dates
        gaze2_num_all_dates = (owgaze2_num_all_dates + mtgaze2_num_all_dates)/session_recordtime_all_dates
        #
        gazeotherlever1_num_all_dates = gazeotherlever1_num_all_dates/session_recordtime_all_dates
        gazeotherlever2_num_all_dates = gazeotherlever2_num_all_dates/session_recordtime_all_dates
        
        #
        # 100: self; 3: 3s coop; 2: 2s coop; 1.5: 1.5s coop; 1: 1s coop; -1: no-vision
        tasktypes_all_dates[tasktypes_all_dates==5] = -1 # change the task type code for no-vision
        coopthres_forsort = (tasktypes_all_dates-1)*coopthres_all_dates/2
        coopthres_forsort[coopthres_forsort==0] = 0 # get the cooperation threshold for sorting

        if grouptype == 'self autolever':
            # 
            gazenum_foreachgroup_foreachAni[grouptype][animal1] = gaze1_num_all_dates[coopthres_forsort==3.5]
            gazenum_foreachgroup_foreachAni[grouptype][animal2] = gaze2_num_all_dates[coopthres_forsort==4]
            #
            gazeothernum_foreachgroup_foreachAni[grouptype][animal1] = gazeotherlever1_num_all_dates[coopthres_forsort==3.5]
            gazeothernum_foreachgroup_foreachAni[grouptype][animal2] = gazeotherlever2_num_all_dates[coopthres_forsort==4]
        #
        elif grouptype == 'other autolever':
            # 
            gazenum_foreachgroup_foreachAni[grouptype][animal1] = gaze1_num_all_dates[coopthres_forsort==4]
            gazenum_foreachgroup_foreachAni[grouptype][animal2] = gaze2_num_all_dates[coopthres_forsort==3.5]
            #
            gazeothernum_foreachgroup_foreachAni[grouptype][animal1] = gazeotherlever1_num_all_dates[coopthres_forsort==4]
            gazeothernum_foreachgroup_foreachAni[grouptype][animal2] = gazeotherlever2_num_all_dates[coopthres_forsort==3.5]
        #
        else:
            # 
            gazenum_foreachgroup_foreachAni[grouptype][animal1] = gaze1_num_all_dates[coopthres_forsort==coopthres_ID]
            gazenum_foreachgroup_foreachAni[grouptype][animal2] = gaze2_num_all_dates[coopthres_forsort==coopthres_ID]
            #
            gazeothernum_foreachgroup_foreachAni[grouptype][animal1] = gazeotherlever1_num_all_dates[coopthres_forsort==coopthres_ID]
            gazeothernum_foreachgroup_foreachAni[grouptype][animal2] = gazeotherlever2_num_all_dates[coopthres_forsort==coopthres_ID]
    
    
    # combine across all animals
    gazenum_foreachgroup_all[grouptype] = np.hstack(list(gazenum_foreachgroup_foreachAni[grouptype].values()))
    gazeothernum_foreachgroup_all[grouptype] = np.hstack(list(gazeothernum_foreachgroup_foreachAni[grouptype].values()))
    
    # combine across male and female
    df = pd.DataFrame.from_dict(gazenum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[malenames]
    gazenum_foreachgroup_male[grouptype] = df.values.ravel()
    #
    df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[malenames]
    gazeothernum_foreachgroup_male[grouptype] = df.values.ravel()
    
    df = pd.DataFrame.from_dict(gazenum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[femalenames]
    gazenum_foreachgroup_female[grouptype] = df.values.ravel()
    #
    df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[femalenames]
    gazeothernum_foreachgroup_female[grouptype] = df.values.ravel()
    
    # combine across sub and dom
    df = pd.DataFrame.from_dict(gazenum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[subnames]
    gazenum_foreachgroup_sub[grouptype] = df.values.ravel()
    #
    df = pd.DataFrame.from_dict(gazenum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[domnames]
    gazenum_foreachgroup_dom[grouptype] = df.values.ravel()
    
    df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[subnames]
    gazeothernum_foreachgroup_sub[grouptype] = df.values.ravel()
    #
    df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[domnames]
    gazeothernum_foreachgroup_dom[grouptype] = df.values.ravel()
    
    
        
# box plot 
fig, axs = plt.subplots(4,1)
fig.set_figheight(5*4)
fig.set_figwidth(3*2)

# subplot 1 - all animals
gazenum_foreachgroup_all_df = pd.DataFrame.from_dict(gazenum_foreachgroup_all,orient='index')
gazenum_foreachgroup_all_df = gazenum_foreachgroup_all_df.transpose()
gazenum_foreachgroup_all_df['type'] = 'all social gaze'
gazeothernum_foreachgroup_all_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_all,orient='index')
gazeothernum_foreachgroup_all_df = gazeothernum_foreachgroup_all_df.transpose()
gazeothernum_foreachgroup_all_df['type'] = 'all other lever gaze'
#
df_long=pd.concat([gazenum_foreachgroup_all_df, gazeothernum_foreachgroup_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.boxplot(ax=axs[0],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[0],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[0].set_xlabel('')
axs[0].set_xticklabels('')
axs[0].xaxis.set_tick_params(labelsize=15)
axs[0].set_ylabel("gaze number per second",fontsize=15)
axs[0].set_title('all animals' ,fontsize=24)
axs[0].set_ylim([-0.1,1.7])
axs[0].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')

# subplot 2 - all animals other lever gaze to social gaze ratio
gazenum_foreachgroup_all_df = pd.DataFrame.from_dict(gazenum_foreachgroup_all,orient='index')
gazenum_foreachgroup_all_df = gazenum_foreachgroup_all_df.transpose()
gazeothernum_foreachgroup_all_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_all,orient='index')
gazeothernum_foreachgroup_all_df = gazeothernum_foreachgroup_all_df.transpose()
socialtoleverratio_foreachgroup_all_df = gazeothernum_foreachgroup_all_df/gazenum_foreachgroup_all_df
socialtoleverratio_foreachgroup_all_df['type'] = 'lever gaze to social gaze ratio'
#
df_long=pd.concat([socialtoleverratio_foreachgroup_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
# seaborn.boxplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type')
seaborn.violinplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[1].set_xlabel('')
axs[1].set_xticklabels('')
axs[1].xaxis.set_tick_params(labelsize=15)
axs[1].set_ylabel("lever gaze to social gaze ratio",fontsize=15)
axs[1].set_title('all animals' ,fontsize=24)
axs[1].set_ylim([-0.1,5])
axs[1].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')


# subplot 3 - male and female animals
gazenum_foreachgroup_male_df = pd.DataFrame.from_dict(gazenum_foreachgroup_male,orient='index')
gazenum_foreachgroup_male_df = gazenum_foreachgroup_male_df.transpose()
gazenum_foreachgroup_female_df = pd.DataFrame.from_dict(gazenum_foreachgroup_female,orient='index')
gazenum_foreachgroup_female_df = gazenum_foreachgroup_female_df.transpose()
#
gazeothernum_foreachgroup_male_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_male,orient='index')
gazeothernum_foreachgroup_male_df = gazeothernum_foreachgroup_male_df.transpose()
gazeothernum_foreachgroup_female_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_female,orient='index')
gazeothernum_foreachgroup_female_df = gazeothernum_foreachgroup_female_df.transpose()
# 
socialtoleverratio_foreachgroup_male_df = gazeothernum_foreachgroup_male_df/gazenum_foreachgroup_male_df
socialtoleverratio_foreachgroup_male_df['type'] = 'male lever gaze to social gaze ratio'
socialtoleverratio_foreachgroup_female_df = gazeothernum_foreachgroup_female_df/gazenum_foreachgroup_female_df
socialtoleverratio_foreachgroup_female_df['type'] = 'female lever gaze to social gaze ratio'
#
gazenum_foreachgroup_male_df['type'] = 'male social gaze'
gazenum_foreachgroup_female_df['type'] = 'female social gaze'
gazeothernum_foreachgroup_male_df['type'] = 'male other lever gaze'
gazeothernum_foreachgroup_female_df['type'] = 'female other lever gaze'
#
# df_long=pd.concat([gazenum_foreachgroup_male_df,gazenum_foreachgroup_female_df,
#                    gazeothernum_foreachgroup_male_df,gazeothernum_foreachgroup_female_df])
df_long=pd.concat([socialtoleverratio_foreachgroup_male_df,
                   socialtoleverratio_foreachgroup_female_df,
                   ])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.violinplot(ax=axs[2],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[2],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[2].set_xlabel('')
axs[2].set_xticklabels('')
axs[2].xaxis.set_tick_params(labelsize=15)
axs[2].set_ylabel("gaze number per second",fontsize=15)
axs[2].set_title('male and female' ,fontsize=24)
axs[2].set_ylim([-0.1,5])
axs[2].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')

# subplot 4 - dom and sub animals
gazenum_foreachgroup_sub_df = pd.DataFrame.from_dict(gazenum_foreachgroup_sub,orient='index')
gazenum_foreachgroup_sub_df = gazenum_foreachgroup_sub_df.transpose()
gazenum_foreachgroup_dom_df = pd.DataFrame.from_dict(gazenum_foreachgroup_dom,orient='index')
gazenum_foreachgroup_dom_df = gazenum_foreachgroup_dom_df.transpose()
#
gazeothernum_foreachgroup_sub_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_sub,orient='index')
gazeothernum_foreachgroup_sub_df = gazeothernum_foreachgroup_sub_df.transpose()
gazeothernum_foreachgroup_dom_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_dom,orient='index')
gazeothernum_foreachgroup_dom_df = gazeothernum_foreachgroup_dom_df.transpose()
#
socialtoleverratio_foreachgroup_sub_df = gazeothernum_foreachgroup_sub_df/gazenum_foreachgroup_sub_df
socialtoleverratio_foreachgroup_sub_df['type'] = 'sub lever gaze to social gaze ratio'
socialtoleverratio_foreachgroup_dom_df = gazeothernum_foreachgroup_dom_df/gazenum_foreachgroup_dom_df
socialtoleverratio_foreachgroup_dom_df['type'] = 'dom lever gaze to social gaze ratio'
#
gazenum_foreachgroup_sub_df['type'] = 'sub social gaze'
gazenum_foreachgroup_dom_df['type'] = 'dom social gaze'
gazeothernum_foreachgroup_sub_df['type'] = 'sub other lever gaze'
gazeothernum_foreachgroup_dom_df['type'] = 'dom other lever gaze'
#
# df_long=pd.concat([gazenum_foreachgroup_sub_df,gazenum_foreachgroup_dom_df,
#                    gazeothernum_foreachgroup_sub_df,gazeothernum_foreachgroup_dom_df,])
df_long=pd.concat([socialtoleverratio_foreachgroup_sub_df,
                   socialtoleverratio_foreachgroup_dom_df,
                   ])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.violinplot(ax=axs[3],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[3],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[3].set_xlabel('')
axs[3].set_xticklabels(axs[3].get_xticklabels(),rotation=45)
axs[3].xaxis.set_tick_params(labelsize=15)
axs[3].set_ylabel("gaze number per second",fontsize=15)
axs[3].set_title('sub and dom' ,fontsize=24)
axs[3].set_ylim([-0.1,5.7])
axs[3].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')

        
savefigs = 0
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+'averaged_gazenumbers_andotherlevernumbers_acrossAllAnimals.pdf')    


#### significance check using anova

In [ ]:
gazenum_foreachgroup_all_df = pd.DataFrame.from_dict(gazenum_foreachgroup_all,orient='index')
gazenum_foreachgroup_all_df = gazenum_foreachgroup_all_df.transpose()
gazeothernum_foreachgroup_all_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_all,orient='index')
gazeothernum_foreachgroup_all_df = gazeothernum_foreachgroup_all_df.transpose()
socialtoleverratio_foreachgroup_all_df = gazeothernum_foreachgroup_all_df/gazenum_foreachgroup_all_df
gazenum_foreachgroup_all_df['type'] = 'all social gaze'
gazeothernum_foreachgroup_all_df['type'] = 'all other lever gaze'
socialtoleverratio_foreachgroup_all_df['type'] = 'lever gaze to social gaze ratio'
#
df_long=pd.concat([gazenum_foreachgroup_all_df, gazeothernum_foreachgroup_all_df,
                   socialtoleverratio_foreachgroup_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
df_long2 = df_long2[~np.isnan(df_long2['value'])]

In [ ]:
xxx = np.array(df_long[df_long['type']=='lever gaze to social gaze ratio']['1s threshold'])
xxx = xxx[~np.isnan(xxx)]
yyy = np.array(df_long[df_long['type']=='lever gaze to social gaze ratio']['other autolever'])
yyy = yyy[~np.isnan(yyy)]

st.ttest_ind(xxx, yyy)

In [ ]:
xxx = np.array(df_long[df_long['type']=='all social gaze']['self reward'])
xxx = xxx[~np.isnan(xxx)]
yyy = np.array(df_long[df_long['type']=='all social gaze']['1s threshold'])
yyy = yyy[~np.isnan(yyy)]

st.ttest_ind(xxx, yyy)

In [ ]:
xxx = np.array(df_long[df_long['type']=='all social gaze']['other autolever'])
xxx = xxx[~np.isnan(xxx)]
yyy = np.array(df_long[df_long['type']=='all other lever gaze']['other autolever'])
yyy = yyy[~np.isnan(yyy)]

st.ttest_ind(xxx, yyy)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# anova
cw_lm=ols('value ~ type + condition + type:condition', data=df_long2).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))
# post hoc test 
tukey = pairwise_tukeyhsd(endog=df_long2['value'], groups=df_long2['type']+df_long2['condition'], alpha=0.05)
print(tukey)

In [ ]:
gazenum_foreachgroup_male_df = pd.DataFrame.from_dict(gazenum_foreachgroup_male,orient='index')
gazenum_foreachgroup_male_df = gazenum_foreachgroup_male_df.transpose()
gazenum_foreachgroup_male_df['type'] = 'social gaze'
gazenum_foreachgroup_male_df['sex'] = 'male'

gazenum_foreachgroup_female_df = pd.DataFrame.from_dict(gazenum_foreachgroup_female,orient='index')
gazenum_foreachgroup_female_df = gazenum_foreachgroup_female_df.transpose()
gazenum_foreachgroup_female_df['type'] = 'social gaze'
gazenum_foreachgroup_female_df['sex'] = 'female'

gazeothernum_foreachgroup_male_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_male,orient='index')
gazeothernum_foreachgroup_male_df = gazeothernum_foreachgroup_male_df.transpose()
gazeothernum_foreachgroup_male_df['type'] = 'other lever gaze'
gazeothernum_foreachgroup_male_df['sex'] = 'male'

gazeothernum_foreachgroup_female_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_female,orient='index')
gazeothernum_foreachgroup_female_df = gazeothernum_foreachgroup_female_df.transpose()
gazeothernum_foreachgroup_female_df['type'] = 'other lever gaze'
gazeothernum_foreachgroup_female_df['sex'] = 'female'

#
df_long=pd.concat([gazenum_foreachgroup_male_df,gazenum_foreachgroup_female_df,
                   gazeothernum_foreachgroup_male_df,gazeothernum_foreachgroup_female_df])
df_long2 = df_long.melt(id_vars=['type','sex'], value_vars=grouptypes,var_name='condition', value_name='value')
df_long2 = df_long2[~np.isnan(df_long2['value'])]

# perform the anova on male and female
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# anova
cw_lm=ols('value ~ type + sex + condition', data=df_long2).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))
# post hoc test 
tukey = pairwise_tukeyhsd(endog=df_long2['value'], groups=df_long2['type']+df_long2['sex']+df_long2['condition'], alpha=0.05)
print(tukey)


In [ ]:
gazenum_foreachgroup_all_df = pd.DataFrame.from_dict(gazenum_foreachgroup_all,orient='index')
gazenum_foreachgroup_all_df = gazenum_foreachgroup_all_df.transpose()
gazeothernum_foreachgroup_all_df = pd.DataFrame.from_dict(gazeothernum_foreachgroup_all,orient='index')
gazeothernum_foreachgroup_all_df = gazeothernum_foreachgroup_all_df.transpose()
socialtoleverratio_foreachgroup_all_df = gazeothernum_foreachgroup_all_df/gazenum_foreachgroup_all_df
socialtoleverratio_foreachgroup_all_df['type'] = 'lever gaze to social gaze ratio'
#
df_long=pd.concat([socialtoleverratio_foreachgroup_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
#
df_long2 = df_long2[~np.isnan(df_long2['value'])]

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#
# anova
cw_lm=ols('value ~ condition', data=df_long2).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))
# post hoc test 
tukey = pairwise_tukeyhsd(endog=df_long2['value'], groups=df_long2['condition'], alpha=0.05)
print(tukey)

In [ ]:
# male female
#df_long=pd.concat([socialtoleverratio_foreachgroup_male_df,
#                    socialtoleverratio_foreachgroup_female_df,
#                   ])
df_long=pd.concat([socialtoleverratio_foreachgroup_sub_df,
                   socialtoleverratio_foreachgroup_dom_df,
                   ])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
#
df_long2 = df_long2[~np.isnan(df_long2['value'])]

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#
# anova
cw_lm=ols('value ~ condition+type', data=df_long2).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))
# post hoc test 
tukey = pairwise_tukeyhsd(endog=df_long2['value'], groups=df_long2['condition']+df_long2['type'], alpha=0.05)
print(tukey)

In [ ]:
# data1 = np.array(df_long[df_long['type']=='male lever gaze to social gaze ratio']['other autolever'])
# data2 = np.array(df_long[df_long['type']=='female lever gaze to social gaze ratio']['other autolever'])
data1 = np.array(df_long[df_long['type']=='sub lever gaze to social gaze ratio']['other autolever'])
data2 = np.array(df_long[df_long['type']=='dom lever gaze to social gaze ratio']['other autolever'])
data1=data1[~np.isnan(data1)]
data2=data2[~np.isnan(data2)]

st.mannwhitneyu(data1,data2)


In [ ]:
data2

### similar plot as the previous one comparing gaze to other's lever vs face
### compare the ratio of each conditions vs. MC (MC ratio as the baseline)

In [ ]:
animal1_fixedorders = ['eddie','dodson','dannon','ginger']
animal2_fixedorders = ['sparkle','scorch','kanga_1','kanga_2']
# animal1_fixedorders = ['eddie','dodson','ginger']
# animal2_fixedorders = ['sparkle','scorch','kanga_2']
nanimalpairs = np.shape(animal1_fixedorders)[0]

# grouptypes = ['self reward','1s threshold','animal1 autolever','animal2_autolever']
grouptypes = ['self reward','1s threshold','self autolever','other autolever']
coopthres_IDs = [0, 1, 3.5, 4]
ngrouptypes = np.shape(grouptypes)[0]

gazenum_gazeothernum_summary = pd.DataFrame(columns=['IDs','act_animal','conditions','gazenum','gazeothernum'])
#
malenames = ['eddie','dodson','dannon',]
femalenames = ['sparkle','scorch','kanga_1','kanga_2','ginger',]
#
subnames = ['eddie','dodson','dannon','ginger',]
domnames = ['sparkle','scorch','kanga_1','kanga_2',]

#
for igrouptype in np.arange(0,ngrouptypes,1):
    
    grouptype = grouptypes[igrouptype]
    coopthres_ID = coopthres_IDs[igrouptype]
    
    
    #
    for ianimalpair in np.arange(0,nanimalpairs,1):
        animal1 = animal1_fixedorders[ianimalpair]
        animal2 = animal2_fixedorders[ianimalpair]
        
        if (animal2 == 'kanga_1') | (animal2 == 'kanga_2'):
            animal2_filename = 'kanga'
        else:
            animal2_filename = animal2

        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1+animal2_filename+'/'

        with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            owgaze1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            owgaze2_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            mtgaze1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            mtgaze2_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            pull1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            pull2_num_all_dates = pickle.load(f)

        with open(data_saved_subfolder+'/gazelever1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazelever1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazelever2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazelever2_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazetube1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazetube1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazetube2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazetube2_num_all_dates = pickle.load(f)

        with open(data_saved_subfolder+'/gazeotherlever1_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazeotherlever1_num_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazeotherlever2_num_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazeotherlever2_num_all_dates = pickle.load(f)

        with open(data_saved_subfolder+'/session_recordtime_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            session_recordtime_all_dates = pickle.load(f)
        
        with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            tasktypes_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            coopthres_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            succ_rate_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            interpullintv_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            trialnum_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            bhv_intv_all_dates = pickle.load(f)

        # combine owgaze and mtgaze
        gaze1_num_all_dates = (owgaze1_num_all_dates + mtgaze1_num_all_dates)/session_recordtime_all_dates
        gaze2_num_all_dates = (owgaze2_num_all_dates + mtgaze2_num_all_dates)/session_recordtime_all_dates
        #
        gazeotherlever1_num_all_dates = gazeotherlever1_num_all_dates/session_recordtime_all_dates
        gazeotherlever2_num_all_dates = gazeotherlever2_num_all_dates/session_recordtime_all_dates
        
        #
        # 100: self; 3: 3s coop; 2: 2s coop; 1.5: 1.5s coop; 1: 1s coop; -1: no-vision
        tasktypes_all_dates[tasktypes_all_dates==5] = -1 # change the task type code for no-vision
        coopthres_forsort = (tasktypes_all_dates-1)*coopthres_all_dates/2
        coopthres_forsort[coopthres_forsort==0] = 0 # get the cooperation threshold for sorting

        if grouptype == 'self autolever':
            # 
            gazenum_foreachgroup_foreachAni_animal1 = gaze1_num_all_dates[coopthres_forsort==3.5]
            gazenum_foreachgroup_foreachAni_animal2 = gaze2_num_all_dates[coopthres_forsort==4]
            #
            gazeothernum_foreachgroup_foreachAni_animal1 = gazeotherlever1_num_all_dates[coopthres_forsort==3.5]
            gazeothernum_foreachgroup_foreachAni_animal2 = gazeotherlever2_num_all_dates[coopthres_forsort==4]
        #
        elif grouptype == 'other autolever':
            # 
            gazenum_foreachgroup_foreachAni_animal1 = gaze1_num_all_dates[coopthres_forsort==4]
            gazenum_foreachgroup_foreachAni_animal2 = gaze2_num_all_dates[coopthres_forsort==3.5]
            #
            gazeothernum_foreachgroup_foreachAni_animal1 = gazeotherlever1_num_all_dates[coopthres_forsort==4]
            gazeothernum_foreachgroup_foreachAni_animal2 = gazeotherlever2_num_all_dates[coopthres_forsort==3.5]
        #
        else:
            # 
            gazenum_foreachgroup_foreachAni_animal1 = gaze1_num_all_dates[coopthres_forsort==coopthres_ID]
            gazenum_foreachgroup_foreachAni_animal2 = gaze2_num_all_dates[coopthres_forsort==coopthres_ID]
            #
            gazeothernum_foreachgroup_foreachAni_animal1 = gazeotherlever1_num_all_dates[coopthres_forsort==coopthres_ID]
            gazeothernum_foreachgroup_foreachAni_animal2 = gazeotherlever2_num_all_dates[coopthres_forsort==coopthres_ID]
    
        # put data in the combined pd
        nentry_animal1 = np.shape(gazenum_foreachgroup_foreachAni_animal1)[0]
        for ientry in np.arange(0,nentry_animal1,1):
            gazenum_gazeothernum_summary = gazenum_gazeothernum_summary.append({'IDs':ientry,
                                                                                'act_animal':animal1,
                                                                                'conditions':grouptype,
                                                                                'gazenum':gazenum_foreachgroup_foreachAni_animal1[ientry],
                                                                                'gazeothernum':gazeothernum_foreachgroup_foreachAni_animal1[ientry],},
                                                                                 ignore_index=True)
        nentry_animal2 = np.shape(gazenum_foreachgroup_foreachAni_animal2)[0]
        for ientry in np.arange(0,nentry_animal2,1):
            gazenum_gazeothernum_summary = gazenum_gazeothernum_summary.append({'IDs':ientry,
                                                                                'act_animal':animal2,
                                                                                'conditions':grouptype,
                                                                                'gazenum':gazenum_foreachgroup_foreachAni_animal2[ientry],
                                                                                'gazeothernum':gazeothernum_foreachgroup_foreachAni_animal2[ientry],},
                                                                                 ignore_index=True)
    
gazenum_gazeothernum_summary['sex'] = np.nan
gazenum_gazeothernum_summary['hierarchy'] = np.nan

gazenum_gazeothernum_summary['levertofaceratio'] = gazenum_gazeothernum_summary['gazeothernum']/gazenum_gazeothernum_summary['gazenum']
   
gazenum_gazeothernum_summary['ratio_vs_MC'] = np.nan
    
# box plot 
fig, axs = plt.subplots(4,1)
fig.set_figheight(3*4)
fig.set_figwidth(3*2)

# subplot 1 - all animals
seaborn.violinplot(ax=axs[0],data=gazenum_gazeothernum_summary,x='conditions',y='levertofaceratio')
axs[0].set_ylabel("lever gaze to social gaze ratio")
axs[0].set_title('all animals')
axs[0].set_ylim([-0.1,5])
axs[0].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')

# subplot 2 - all animals, minus MC mean
actanimals = np.unique(gazenum_gazeothernum_summary['act_animal'])
for actanimal in actanimals:
    ind = gazenum_gazeothernum_summary['act_animal'] == actanimal
    gazenum_gazeothernum_ianimal = gazenum_gazeothernum_summary[ind].copy()
    #
    MCmean = np.nanmean(gazenum_gazeothernum_ianimal[gazenum_gazeothernum_ianimal['conditions']=='1s threshold']['levertofaceratio'])
    #
    gazenum_gazeothernum_summary.loc[ind,'ratio_vs_MC'] = gazenum_gazeothernum_ianimal['levertofaceratio']-MCmean
#
seaborn.violinplot(ax=axs[1],data=gazenum_gazeothernum_summary,x='conditions',y='ratio_vs_MC')
axs[1].plot([-0.5,3.5],[0,0],'k--')
axs[1].set_ylabel("ratio; compared with MC mean")
axs[1].set_title('all animals')
axs[1].set_ylim([-3,3])
axs[1].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')    

# subplot 3 - male female, minus MC mean
actanimals = np.unique(gazenum_gazeothernum_summary['act_animal'])
for actanimal in actanimals:
    ind = gazenum_gazeothernum_summary['act_animal'] == actanimal
    #
    if np.isin(actanimal,malenames):
        gazenum_gazeothernum_summary.loc[ind,'sex'] = 'male'
    else:
        gazenum_gazeothernum_summary.loc[ind,'sex'] = 'female'
#
seaborn.violinplot(ax=axs[2],data=gazenum_gazeothernum_summary,x='conditions',y='ratio_vs_MC',hue='sex')
axs[2].plot([-0.5,3.5],[0,0],'k--')
axs[2].set_ylabel("ratio; compared with MC mean")
axs[2].set_title('male female')
axs[2].set_ylim([-3,3])
axs[2].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')  

# subplot 4 - sub dom, minus MC mean
actanimals = np.unique(gazenum_gazeothernum_summary['act_animal'])
for actanimal in actanimals:
    ind = gazenum_gazeothernum_summary['act_animal'] == actanimal
    #
    if np.isin(actanimal,subnames):
        gazenum_gazeothernum_summary.loc[ind,'hierarchy'] = 'sub'
    else:
        gazenum_gazeothernum_summary.loc[ind,'hierarchy'] = 'dom'
#
seaborn.violinplot(ax=axs[3],data=gazenum_gazeothernum_summary,x='conditions',y='ratio_vs_MC',hue='hierarchy')
axs[3].plot([-0.5,3.5],[0,0],'k--')
axs[3].set_ylabel("ratio; compared with MC mean")
axs[3].set_title('sub dom')
axs[3].set_ylim([-3,3])
axs[3].legend(fontsize=18, bbox_to_anchor=(1.05, 1), loc='upper left')  


plt.tight_layout()
        
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+'averaged_gazenumbers_andotherlevernumbers_comparedwithMC_acrossAllAnimals.pdf')    


In [ ]:
st.wilcoxon(gazenum_gazeothernum_summary[gazenum_gazeothernum_summary['conditions']=='other autolever']['ratio_vs_MC'])



In [ ]:
ind1=(gazenum_gazeothernum_summary['conditions']=='other autolever')&(gazenum_gazeothernum_summary['sex']=='male')
data1 = gazenum_gazeothernum_summary[ind1]['ratio_vs_MC']
ind2=(gazenum_gazeothernum_summary['conditions']=='other autolever')&(gazenum_gazeothernum_summary['sex']=='female')
data2 = gazenum_gazeothernum_summary[ind2]['ratio_vs_MC']

st.mannwhitneyu(data1,data2)
# st.ttest_ind(data1,data2)



In [ ]:
ind1=(gazenum_gazeothernum_summary['conditions']=='other autolever')&(gazenum_gazeothernum_summary['hierarchy']=='sub')
data1 = gazenum_gazeothernum_summary[ind1]['ratio_vs_MC']
ind2=(gazenum_gazeothernum_summary['conditions']=='other autolever')&(gazenum_gazeothernum_summary['hierarchy']=='dom')
data2 = gazenum_gazeothernum_summary[ind2]['ratio_vs_MC']

st.mannwhitneyu(data1,data2)
# st.ttest_ind(data1,data2)

#### plot the gaze distribution around pulls, analysis is based on the DBN_input_data all session format
#### similar plot was in "3LagDBN_and_SuccAndFailedPull_singlecam_wholebodylabels_allsessions_basicEvents" looking at the difference between successful and failed pulls
#### pool across all animals, compared self reward, 3s to 1s cooperation and no vision

In [ ]:
# PLOT multiple pairs in one plot, so need to load data seperately
mergetempRos = 0 # 1: merge different time bins
minmaxfullSampSize = 1 # 1: use the  min row number and max row number, or the full row for each session
moreSampSize = 0 # 1: use more sample size (more than just minimal row number and max row number)
#
animal1_fixedorders = ['eddie','dodson','dannon','ginger',]
animal2_fixedorders = ['sparkle','scorch','kanga_1','kanga_2',]
nanimalpairs = np.shape(animal1_fixedorders)[0]

temp_resolu = 1
dist_twin_range = 5

grouptypes = ['self reward','1s threshold','animal1 autolever','animal2 autolever']
# grouptypes = ['self reward','1s threshold','self autolever','other autolever']
coopthres_IDs = [0, 1, 3.5, 4]
ngrouptypes = np.shape(grouptypes)[0]

# initiate the final data set
SameAnimal_gazeDist_mean_forEachAni = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_mean_forEachAni = dict.fromkeys(grouptypes,[])
# shuffle both the pull and gaze time stamp
SameAnimal_gazeDist_shuffle_forEachAni = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_shuffle_forEachAni = dict.fromkeys(grouptypes,[])
#
SameAnimal_gazeDist_mean_all = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_mean_all = dict.fromkeys(grouptypes,[])
# shuffle both the pull and gaze time stamp
SameAnimal_gazeDist_shuffle_all = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_shuffle_all = dict.fromkeys(grouptypes,[])
#
malenames = ['eddie','dodson','dannon',]
femalenames = ['sparkle','scorch','kanga_1','kanga_2','ginger',]
SameAnimal_gazeDist_mean_male = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_mean_male = dict.fromkeys(grouptypes,[])
SameAnimal_gazeDist_mean_female = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_mean_female = dict.fromkeys(grouptypes,[])
#
subnames = ['eddie','dodson','dannon','ginger']
domnames = ['sparkle','scorch','kanga_1','kanga_2']
SameAnimal_gazeDist_mean_sub = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_mean_sub = dict.fromkeys(grouptypes,[])
SameAnimal_gazeDist_mean_dom = dict.fromkeys(grouptypes,[])
AcroAnimal_gazeDist_mean_dom = dict.fromkeys(grouptypes,[])




#
for igrouptype in np.arange(0,ngrouptypes,1):
    
    grouptype = grouptypes[igrouptype]
    coopthres_ID = coopthres_IDs[igrouptype]
    
    SameAnimal_gazeDist_mean_forEachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])
    AcroAnimal_gazeDist_mean_forEachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])
    SameAnimal_gazeDist_shuffle_forEachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])
    AcroAnimal_gazeDist_shuffle_forEachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])   
    
    for ianimalpair in np.arange(0,nanimalpairs,1):
        animal1_fixedorder = animal1_fixedorders[ianimalpair]
        animal2_fixedorder = animal2_fixedorders[ianimalpair]

        if (animal2_fixedorder == 'kanga_1') | (animal2_fixedorder == 'kanga_2'):
            animal2_filename = 'kanga'
        else:
            animal2_filename = animal2_fixedorder
            
        # load the basic behavioral measures
        # load saved data
        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1_fixedorder+animal2_filename+'/'
        #
        with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1_fixedorder+animal2_filename+'.pkl', 'rb') as f:
            tasktypes_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1_fixedorder+animal2_filename+'.pkl', 'rb') as f:
            coopthres_all_dates = pickle.load(f)

        #     
        # load the DBN related analysis
        # load data
        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody_allsessions'+savefile_sufix+'_3lags/'+cameraID+'/'+animal1_fixedorder+animal2_filename+'/'
        #
        if not mergetempRos:
            with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder+animal2_filename+'_'+str(temp_resolu)+'sReSo.pkl', 'rb') as f:
                DBN_input_data_alltypes = pickle.load(f)
        else:
            with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder+animal2_filename+'_mergeTempsReSo.pkl', 'rb') as f:
                DBN_input_data_alltypes = pickle.load(f)

        #
        # re-organize the target dates
        # 100: self; 3: 3s coop; 2: 2s coop; 1.5: 1.5s coop; 1: 1s coop; -1: no-vision
        tasktypes_all_dates[tasktypes_all_dates==5] = -1 # change the task type code for no-vision
        coopthres_forsort = (tasktypes_all_dates-1)*coopthres_all_dates/2
        coopthres_forsort[coopthres_forsort==0] = 0 # get the cooperation threshold for sorting


        #
        # sort the data based on task type and dates
        dates_list = list(DBN_input_data_alltypes.keys())
        sorting_df = pd.DataFrame({'dates': dates_list, 'coopthres': coopthres_forsort.ravel()}, columns=['dates', 'coopthres'])
        sorting_df = sorting_df.sort_values(by=['coopthres','dates'], ascending = [False, True])
        #
        # only select the targeted dates
        sorting_tgt_df = sorting_df[(sorting_df['coopthres']==coopthres_ID)]
        dates_list_tgt = sorting_tgt_df['dates']
        dates_list_tgt = np.array(dates_list_tgt)
        #
        ndates_tgt = np.shape(dates_list_tgt)[0]

        #
        # initiate the final data set
        SameAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        SameAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        #
        SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder] = dict.fromkeys(dates_list_tgt,[])
        AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder] = dict.fromkeys(dates_list_tgt,[])

        # 
        for idate in np.arange(0,ndates_tgt,1):
            idate_name = dates_list_tgt[idate]

            DBN_input_data_idate = DBN_input_data_alltypes[idate_name]
            
            # pull1_t0 and gaze1_t0
            xxx1 = (np.array(DBN_input_data_idate['pull1_t0'])==1)*1
            xxx2 = (np.array(DBN_input_data_idate['lvgaze1_t0'])==1)*1
            # xxx2 = (np.array(DBN_input_data_idate['owgaze1_t0'])==1)*1
            xxx1_shuffle = xxx1.copy()
            np.random.shuffle(xxx1_shuffle)
            xxx2_shuffle = xxx2.copy()
            np.random.shuffle(xxx2_shuffle)
            # pad the two sides
            xxx1 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1,np.zeros((1,dist_twin_range))[0]])
            xxx2 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2,np.zeros((1,dist_twin_range))[0]])
            xxx1_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1_shuffle,np.zeros((1,dist_twin_range))[0]])
            xxx2_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2_shuffle,np.zeros((1,dist_twin_range))[0]])
            # 
            npulls = int(np.nansum(xxx1))
            pullIDs = np.where(xxx1 == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            SameAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # SameAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))
            if npulls == 0:
                SameAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan        
            # shuffle
            npulls = int(np.nansum(xxx1_shuffle))
            pullIDs = np.where(xxx1_shuffle == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2_shuffle[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))           
            if npulls == 0:
                SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan        
                
            # pull2_t0 and gaze2_t0
            xxx1 = (np.array(DBN_input_data_idate['pull2_t0'])==1)*1
            xxx2 = (np.array(DBN_input_data_idate['lvgaze2_t0'])==1)*1
            # xxx2 = (np.array(DBN_input_data_idate['owgaze2_t0'])==1)*1
            xxx1_shuffle = xxx1.copy()
            np.random.shuffle(xxx1_shuffle)
            xxx2_shuffle = xxx2.copy()
            np.random.shuffle(xxx2_shuffle)
            # pad the two sides
            xxx1 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1,np.zeros((1,dist_twin_range))[0]])
            xxx2 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2,np.zeros((1,dist_twin_range))[0]])
            xxx1_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1_shuffle,np.zeros((1,dist_twin_range))[0]])
            xxx2_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2_shuffle,np.zeros((1,dist_twin_range))[0]])
            # 
            npulls = int(np.nansum(xxx1))
            pullIDs = np.where(xxx1 == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            SameAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # SameAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))            
            if npulls == 0:
                SameAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan 
            # shuffle
            npulls = int(np.nansum(xxx1_shuffle))
            pullIDs = np.where(xxx1_shuffle == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2_shuffle[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))
            if npulls == 0:
                SameAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan 
                
            # pull1_t0 and gaze2_t0
            xxx1 = (np.array(DBN_input_data_idate['pull1_t0'])==1)*1
            xxx2 = (np.array(DBN_input_data_idate['lvgaze2_t0'])==1)*1
            # xxx2 = (np.array(DBN_input_data_idate['owgaze2_t0'])==1)*1
            xxx1_shuffle = xxx1.copy()
            np.random.shuffle(xxx1_shuffle)
            xxx2_shuffle = xxx2.copy()
            np.random.shuffle(xxx2_shuffle)
            # pad the two sides
            xxx1 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1,np.zeros((1,dist_twin_range))[0]])
            xxx2 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2,np.zeros((1,dist_twin_range))[0]])
            xxx1_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1_shuffle,np.zeros((1,dist_twin_range))[0]])
            xxx2_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2_shuffle,np.zeros((1,dist_twin_range))[0]])
            # 
            npulls = int(np.nansum(xxx1))
            pullIDs = np.where(xxx1 == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))
            if npulls == 0:
                AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan 
            # shuffle
            npulls = int(np.nansum(xxx1_shuffle))
            pullIDs = np.where(xxx1_shuffle == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2_shuffle[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))
            if npulls == 0:
                AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal2_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan 
    
            # pull2_t0 and gaze1_t0
            xxx1 = (np.array(DBN_input_data_idate['pull2_t0'])==1)*1
            xxx2 = (np.array(DBN_input_data_idate['lvgaze1_t0'])==1)*1
            # xxx2 = (np.array(DBN_input_data_idate['owgaze1_t0'])==1)*1
            xxx1_shuffle = xxx1.copy()
            np.random.shuffle(xxx1_shuffle)
            xxx2_shuffle = xxx2.copy()
            np.random.shuffle(xxx2_shuffle)
            # pad the two sides
            xxx1 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1,np.zeros((1,dist_twin_range))[0]])
            xxx2 = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2,np.zeros((1,dist_twin_range))[0]])
            xxx1_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx1_shuffle,np.zeros((1,dist_twin_range))[0]])
            xxx2_shuffle = np.hstack([np.zeros((1,dist_twin_range))[0],xxx2_shuffle,np.zeros((1,dist_twin_range))[0]])
            # 
            npulls = int(np.nansum(xxx1))
            pullIDs = np.where(xxx1 == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))
            if npulls == 0:
                AcroAnimal_gazeDist_mean_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan 
            # shuffle
            npulls = int(np.nansum(xxx1_shuffle))
            pullIDs = np.where(xxx1_shuffle == 1)[0]
            gazenum_dist_temp = np.zeros((npulls,2*dist_twin_range+1))
            #
            for ipull in np.arange(0,npulls,1):
                pullID = pullIDs[ipull]
                gazenum_dist_temp[ipull,:] = xxx2_shuffle[np.arange(pullID-dist_twin_range,pullID+dist_twin_range+1,1)]
            AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nanmean(gazenum_dist_temp,axis=0)/(np.sum(xxx2)/np.sum(xxx1))
            # AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.nansum(gazenum_dist_temp,axis=0)# /(np.sum(xxx2))
            if npulls == 0:
                AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][animal1_fixedorder][idate_name]=np.ones((1,2*dist_twin_range+1))[0]*np.nan 

    
    # combine across all animals
    df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in animal1_fixedorders+animal2_fixedorders])
    SameAnimal_gazeDist_mean_all[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in animal1_fixedorders+animal2_fixedorders])
    AcroAnimal_gazeDist_mean_all[grouptype] = np.vstack(df.stack().values)
    
    # combine across al animals for shuffle
    df = pd.DataFrame([SameAnimal_gazeDist_shuffle_forEachAni[grouptype][name] for name in animal1_fixedorders+animal2_fixedorders])
    SameAnimal_gazeDist_shuffle_all[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([AcroAnimal_gazeDist_shuffle_forEachAni[grouptype][name] for name in animal1_fixedorders+animal2_fixedorders])
    AcroAnimal_gazeDist_shuffle_all[grouptype] = np.vstack(df.stack().values)
    
    # combine across male and female
    df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in malenames])
    SameAnimal_gazeDist_mean_male[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in malenames])
    AcroAnimal_gazeDist_mean_male[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in femalenames])
    SameAnimal_gazeDist_mean_female[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in femalenames])
    AcroAnimal_gazeDist_mean_female[grouptype] = np.vstack(df.stack().values)
    
    # combine across sub and dom
    df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in subnames])
    SameAnimal_gazeDist_mean_sub[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in subnames])
    AcroAnimal_gazeDist_mean_sub[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in domnames])
    SameAnimal_gazeDist_mean_dom[grouptype] = np.vstack(df.stack().values)
    df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni[grouptype][name] for name in domnames])
    AcroAnimal_gazeDist_mean_dom[grouptype] = np.vstack(df.stack().values)

    
# add the self autolever and other autolever
SameAnimal_gazeDist_mean_forEachAni['self autolever'] = {
                     'eddie':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['eddie'],
                     'dodson':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['dodson'],
                     'dannon':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['dannon'],
                     'ginger':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['ginger'],
                     'sparkle':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['sparkle'],
                     'scorch':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['scorch'],
                     'kanga_1':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['kanga_1'],
                     'kanga_2':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['kanga_2'],
                    }
df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni['self autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
SameAnimal_gazeDist_mean_all['self autolever'] = np.vstack(df.stack().values)
#
SameAnimal_gazeDist_mean_forEachAni['other autolever'] = {
                     'eddie':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['eddie'],
                     'dodson':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['dodson'],
                     'dannon':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['dannon'],
                     'ginger':SameAnimal_gazeDist_mean_forEachAni['animal2 autolever']['ginger'],
                     'sparkle':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['sparkle'],
                     'scorch':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['scorch'],
                     'kanga_1':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['kanga_1'],
                     'kanga_2':SameAnimal_gazeDist_mean_forEachAni['animal1 autolever']['kanga_2'],
                    }
df = pd.DataFrame([SameAnimal_gazeDist_mean_forEachAni['other autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
SameAnimal_gazeDist_mean_all['other autolever'] = np.vstack(df.stack().values)
#
AcroAnimal_gazeDist_mean_forEachAni['self autolever'] = {
                     'eddie':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['eddie'],
                     'dodson':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['dodson'],
                     'dannon':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['dannon'],
                     'ginger':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['ginger'],
                     'sparkle':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['sparkle'],
                     'scorch':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['scorch'],
                     'kanga_1':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['kanga_1'],
                     'kanga_2':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['kanga_2'],
                    }
df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni['self autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
AcroAnimal_gazeDist_mean_all['self autolever'] = np.vstack(df.stack().values)
#
AcroAnimal_gazeDist_mean_forEachAni['other autolever'] = {
                     'eddie':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['eddie'],
                     'dodson':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['dodson'],
                     'dannon':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['dannon'],
                     'ginger':AcroAnimal_gazeDist_mean_forEachAni['animal2 autolever']['ginger'],
                     'sparkle':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['sparkle'],
                     'scorch':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['scorch'],
                     'kanga_1':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['kanga_1'],
                     'kanga_2':AcroAnimal_gazeDist_mean_forEachAni['animal1 autolever']['kanga_2'],
                    }
df = pd.DataFrame([AcroAnimal_gazeDist_mean_forEachAni['other autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
AcroAnimal_gazeDist_mean_all['other autolever'] = np.vstack(df.stack().values)
#   
SameAnimal_gazeDist_shuffle_forEachAni['self autolever'] = {
                     'eddie':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['eddie'],
                     'dodson':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['dodson'],
                     'dannon':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['dannon'],
                     'ginger':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['ginger'],
                     'sparkle':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['sparkle'],
                     'scorch':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['scorch'],
                     'kanga_1':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['kanga_1'],
                     'kanga_2':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['kanga_2'],
                    }
df = pd.DataFrame([SameAnimal_gazeDist_shuffle_forEachAni['self autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
SameAnimal_gazeDist_shuffle_all['self autolever'] = np.vstack(df.stack().values)
#
SameAnimal_gazeDist_shuffle_forEachAni['other autolever'] = {
                     'eddie':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['eddie'],
                     'dodson':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['dodson'],
                     'dannon':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['dannon'],
                     'ginger':SameAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['ginger'],
                     'sparkle':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['sparkle'],
                     'scorch':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['scorch'],
                     'kanga_1':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['kanga_1'],
                     'kanga_2':SameAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['kanga_2'],
                    }
df = pd.DataFrame([SameAnimal_gazeDist_shuffle_forEachAni['other autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
SameAnimal_gazeDist_shuffle_all['other autolever'] = np.vstack(df.stack().values)
#
AcroAnimal_gazeDist_shuffle_forEachAni['self autolever'] = {
                     'eddie':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['eddie'],
                     'dodson':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['dodson'],
                     'dannon':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['dannon'],
                     'ginger':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['ginger'],
                     'sparkle':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['sparkle'],
                     'scorch':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['scorch'],
                     'kanga_1':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['kanga_1'],
                     'kanga_2':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['kanga_2'],
                    }
df = pd.DataFrame([AcroAnimal_gazeDist_shuffle_forEachAni['self autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
AcroAnimal_gazeDist_shuffle_all['self autolever'] = np.vstack(df.stack().values)
#
AcroAnimal_gazeDist_shuffle_forEachAni['other autolever'] = {
                     'eddie':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['eddie'],
                     'dodson':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['dodson'],
                     'dannon':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['dannon'],
                     'ginger':AcroAnimal_gazeDist_shuffle_forEachAni['animal2 autolever']['ginger'],
                     'sparkle':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['sparkle'],
                     'scorch':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['scorch'],
                     'kanga_1':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['kanga_1'],
                     'kanga_2':AcroAnimal_gazeDist_shuffle_forEachAni['animal1 autolever']['kanga_2'],
                    }
df = pd.DataFrame([AcroAnimal_gazeDist_shuffle_forEachAni['other autolever'][name] for name in animal1_fixedorders+animal2_fixedorders])
AcroAnimal_gazeDist_shuffle_all['other autolever'] = np.vstack(df.stack().values)    
    
    
#
if 1:
    
    xxx = np.arange(-dist_twin_range,dist_twin_range+1,1)
    
    fig, axs = plt.subplots(3, 2)
    fig.set_figheight(5*3)
    fig.set_figwidth(7*2)   
    
    # plot the summarizing figure
    # plot the within animal and across animal distribution
    
    for iplottype in np.arange(0,2,1):
        # 
        # plot, all animals
        # conds_forplot = ['self reward','1s threshold','animal1 autolever','animal2 autolever']
        conds_forplot = ['self reward','1s threshold','self autolever','other autolever']
        # conds_forplot = ['self reward','3s threshold','2s threshold','1.5s threshold','1s threshold','novision']
        gazeDist_average_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_std_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_average_shf_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_std_shf_forplot = dict.fromkeys(conds_forplot,[])
        for cond_forplot in conds_forplot:
            if iplottype == 0:
                gazeDist_average_forplot[cond_forplot] = np.nanmean(SameAnimal_gazeDist_mean_all[cond_forplot],axis=0)
                gazeDist_std_forplot[cond_forplot] = np.nanstd(SameAnimal_gazeDist_mean_all[cond_forplot],axis=0)/np.sqrt(np.shape(SameAnimal_gazeDist_mean_all[cond_forplot])[0])
                #
                gazeDist_average_shf_forplot[cond_forplot] = np.nanmean(SameAnimal_gazeDist_shuffle_all[cond_forplot],axis=0)
                gazeDist_std_shf_forplot[cond_forplot] = np.nanstd(SameAnimal_gazeDist_shuffle_all[cond_forplot],axis=0)/np.sqrt(np.shape(SameAnimal_gazeDist_shuffle_all[cond_forplot])[0])
            elif iplottype == 1:
                gazeDist_average_forplot[cond_forplot] = np.nanmean(AcroAnimal_gazeDist_mean_all[cond_forplot],axis=0)
                gazeDist_std_forplot[cond_forplot] = np.nanstd(AcroAnimal_gazeDist_mean_all[cond_forplot],axis=0)/np.sqrt(np.shape(AcroAnimal_gazeDist_mean_all[cond_forplot])[0])
                #
                gazeDist_average_shf_forplot[cond_forplot] = np.nanmean(AcroAnimal_gazeDist_shuffle_all[cond_forplot],axis=0)
                gazeDist_std_shf_forplot[cond_forplot] = np.nanstd(AcroAnimal_gazeDist_shuffle_all[cond_forplot],axis=0)/np.sqrt(np.shape(AcroAnimal_gazeDist_shuffle_all[cond_forplot])[0])
            #
            axs[0,iplottype].errorbar(xxx,gazeDist_average_forplot[cond_forplot],
                            gazeDist_std_forplot[cond_forplot],label=cond_forplot)
            # axs[0,iplottype].errorbar(xxx,gazeDist_average_shf_forplot[cond_forplot],
            #                 gazeDist_std_shf_forplot[cond_forplot],label="shuffled "+cond_forplot)
        axs[0,iplottype].plot([0,0],[0,1],'--',color='0.5')
        axs[0,iplottype].set_xlim(-dist_twin_range-0.75,dist_twin_range+0.75)
        axs[0,iplottype].set_ylim(0,0.3)
        # axs[0,iplottype].set_xlabel('time (s)',fontsize=15)
        axs[0,iplottype].set_ylabel('other lever gaze distribution',fontsize=15)
        axs[0,iplottype].legend()   
        if iplottype == 0:
            axs[0,iplottype].set_title('within animal: all animals',fontsize=16)   
        elif iplottype == 1:
            axs[0,iplottype].set_title('across animal: all animals',fontsize=16)

        # plot, male and female
        conds_forplot = ['1s threshold']
        gazeDist_average_male_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_std_male_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_average_female_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_std_female_forplot = dict.fromkeys(conds_forplot,[])
        for cond_forplot in conds_forplot:
            if iplottype == 0:
                gazeDist_average_male_forplot[cond_forplot] = np.nanmean(SameAnimal_gazeDist_mean_male[cond_forplot],axis=0)
                gazeDist_std_male_forplot[cond_forplot] = np.nanstd(SameAnimal_gazeDist_mean_male[cond_forplot],axis=0)/np.sqrt(np.shape(SameAnimal_gazeDist_mean_male[cond_forplot])[0])
                #
                gazeDist_average_female_forplot[cond_forplot] = np.nanmean(SameAnimal_gazeDist_mean_female[cond_forplot],axis=0)
                gazeDist_std_female_forplot[cond_forplot] = np.nanstd(SameAnimal_gazeDist_mean_female[cond_forplot],axis=0)/np.sqrt(np.shape(SameAnimal_gazeDist_mean_female[cond_forplot])[0])
            elif iplottype == 1:
                gazeDist_average_male_forplot[cond_forplot] = np.nanmean(AcroAnimal_gazeDist_mean_male[cond_forplot],axis=0)
                gazeDist_std_male_forplot[cond_forplot] = np.nanstd(AcroAnimal_gazeDist_mean_male[cond_forplot],axis=0)/np.sqrt(np.shape(AcroAnimal_gazeDist_mean_male[cond_forplot])[0])
                #
                gazeDist_average_female_forplot[cond_forplot] = np.nanmean(AcroAnimal_gazeDist_mean_female[cond_forplot],axis=0)
                gazeDist_std_female_forplot[cond_forplot] = np.nanstd(AcroAnimal_gazeDist_mean_female[cond_forplot],axis=0)/np.sqrt(np.shape(AcroAnimal_gazeDist_mean_female[cond_forplot])[0])
            #
            axs[1,iplottype].errorbar(xxx,gazeDist_average_male_forplot[cond_forplot],
                            gazeDist_std_male_forplot[cond_forplot],label='male '+cond_forplot)
            axs[1,iplottype].errorbar(xxx,gazeDist_average_female_forplot[cond_forplot],
                            gazeDist_std_female_forplot[cond_forplot],label='female '+cond_forplot)
        axs[1,iplottype].plot([0,0],[0,1],'--',color='0.5')
        axs[1,iplottype].set_xlim(-dist_twin_range-0.75,dist_twin_range+0.75)
        axs[1,iplottype].set_ylim(0,0.3)
        # axs[1,iplottype].set_xlabel('time (s)',fontsize=15)
        axs[1,iplottype].set_ylabel('other lever gaze distribution',fontsize=15)
        axs[1,iplottype].legend()   
        if iplottype == 0:
            axs[1,iplottype].set_title('within animal: male and female',fontsize=16) 
        elif iplottype == 1:
            axs[1,iplottype].set_title('across animal: male and female',fontsize=16) 

        # plot, sub and dom
        conds_forplot = ['1s threshold']
        gazeDist_average_dom_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_std_dom_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_average_sub_forplot = dict.fromkeys(conds_forplot,[])
        gazeDist_std_sub_forplot = dict.fromkeys(conds_forplot,[])
        for cond_forplot in conds_forplot:
            if iplottype == 0:
                gazeDist_average_sub_forplot[cond_forplot] = np.nanmean(SameAnimal_gazeDist_mean_sub[cond_forplot],axis=0)
                gazeDist_std_sub_forplot[cond_forplot] = np.nanstd(SameAnimal_gazeDist_mean_sub[cond_forplot],axis=0)/np.sqrt(np.shape(SameAnimal_gazeDist_mean_sub[cond_forplot])[0]) 
                #
                gazeDist_average_dom_forplot[cond_forplot] = np.nanmean(SameAnimal_gazeDist_mean_dom[cond_forplot],axis=0)
                gazeDist_std_dom_forplot[cond_forplot] = np.nanstd(SameAnimal_gazeDist_mean_dom[cond_forplot],axis=0)/np.sqrt(np.shape(SameAnimal_gazeDist_mean_dom[cond_forplot])[0])
            elif iplottype == 1:
                gazeDist_average_sub_forplot[cond_forplot] = np.nanmean(AcroAnimal_gazeDist_mean_sub[cond_forplot],axis=0)
                gazeDist_std_sub_forplot[cond_forplot] = np.nanstd(AcroAnimal_gazeDist_mean_sub[cond_forplot],axis=0)/np.sqrt(np.shape(AcroAnimal_gazeDist_mean_sub[cond_forplot])[0]) 
                #
                gazeDist_average_dom_forplot[cond_forplot] = np.nanmean(AcroAnimal_gazeDist_mean_dom[cond_forplot],axis=0)
                gazeDist_std_dom_forplot[cond_forplot] = np.nanstd(AcroAnimal_gazeDist_mean_dom[cond_forplot],axis=0)/np.sqrt(np.shape(AcroAnimal_gazeDist_mean_dom[cond_forplot])[0])
            #
            axs[2,iplottype].errorbar(xxx,gazeDist_average_sub_forplot[cond_forplot],
                            gazeDist_std_sub_forplot[cond_forplot],label='sub '+cond_forplot)
            axs[2,iplottype].errorbar(xxx,gazeDist_average_dom_forplot[cond_forplot],
                            gazeDist_std_dom_forplot[cond_forplot],label='dom '+cond_forplot)
        axs[2,iplottype].plot([0,0],[0,1],'--',color='0.5')
        axs[2,iplottype].set_xlim(-dist_twin_range-0.75,dist_twin_range+0.75)
        axs[2,iplottype].set_ylim(0,0.3)
        axs[2,iplottype].set_xlabel('time (s)',fontsize=15)
        axs[2,iplottype].set_ylabel('other lever gaze distribution',fontsize=15)
        axs[2,iplottype].legend()   
        if iplottype == 0:
            axs[2,iplottype].set_title('within animal: subordinate and dominant',fontsize=16) 
        elif iplottype == 1:
            axs[2,iplottype].set_title('across animal: subordinate and dominant',fontsize=16) 

    savefigs = 1
    if savefigs:
        figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'
        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        plt.savefig(figsavefolder+"socialgaze_distribution_summaryplot.pdf")

In [ ]:
dates_list

In [ ]:
coopthres_forsort

#### get the half (max - min) width for selected conditions 

In [ ]:
from scipy.interpolate import splrep, sproot, splev
import matplotlib.pyplot as plt 
from scipy.optimize import curve_fit 
from scipy.ndimage import gaussian_filter1d

class MultiplePeaks(Exception): pass
class NoPeaksFound(Exception): pass

def fwhm(x, y, k=10):
    """
    Determine full-with-half-maximum of a peaked set of points, x and y.

    Assumes that there is only one peak present in the datasset.  The function
    uses a spline interpolation of order k.
    """
    
    maxtimepoint = x[y==np.nanmax(y)][0]
    
    half_max = max(y)/2.0
    # half_max = y[round(np.shape(y)[0]/2)-1]
    s = splrep(x, y - half_max, k=k)
    roots = sproot(s)

    if len(roots) > 2:
    #     raise MultiplePeaks("The dataset appears to have multiple peaks, and "
    #             "thus the FWHM can't be determined.")
        # return np.nan
        return abs(roots[1] - roots[0]), maxtimepoint
    elif len(roots) < 2:
    #     raise NoPeaksFound("No proper peaks were found in the data set; likely "
    #             "the dataset is flat (e.g. all zeros).")
        # return np.max(x)-np.min(x)
        return np.nan, np.nan
    else:
        return abs(roots[1] - roots[0]), maxtimepoint
        
        
#
# Define the Gaussian function 
def Gauss(x, A, B): 
    y = A*np.exp(-1*B*x**2) 
    return y 

# Define the Gaussian function
def gaussian(x, A, B, C):
    y = A*np.exp(-1*B*(x-C)**2) 
    return y 

In [ ]:
x =  np.arange(-dist_twin_range,dist_twin_range+1,1)

# conditions = list(AcroAnimal_gazeDist_mean_all.keys())
conditions = ['self reward','1s threshold','self autolever','other autolever']
nconds = np.shape(conditions)[0]

halfwidth_all = dict.fromkeys(conditions)
maxtimepoint_all = dict.fromkeys(conditions)

for icond in np.arange(0,nconds,1):

    condname = conditions[icond]
    
    y_allsess = AcroAnimal_gazeDist_mean_all[condname]
    nsess = np.shape(y_allsess)[0]
    
    halfwidth_all[condname] = np.ones((1,nsess))[0]*np.nan
    maxtimepoint_all[condname] = np.ones((1,nsess))[0]*np.nan
    
    for isess in np.arange(0,nsess,1):
    
        try:
            y =  y_allsess[isess]
            y = (y-np.nanmin(y))/(np.nanmax(y)-np.nanmin(y))     
            
            # y = gaussian_filter1d(y, 4)

            maxtimepoint_all[condname][isess] = x[y==np.nanmax(y)][0]
            
            parameters, covariance = curve_fit(Gauss, x, y) 
            # parameters, covariance = curve_fit(gaussian, x, y) 
            #
            fit_A = parameters[0] 
            fit_B = parameters[1] 
            # fit_C = parameters[2] 
            #
            fit_y = Gauss(x, fit_A, fit_B) 
            # fit_y = gaussian(x,fit_A,fit_B,fit_C)
            y = (fit_y-np.nanmin(fit_y))/(np.nanmax(fit_y)-np.nanmin(fit_y)) 
            

            halfwidth_all[condname][isess], _ = fwhm(x, y, k=3)
            # halfwidth_all[condname][isess], maxtimepoint_all[condname][isess] = fwhm(x, y, k=3)
            
        except:
            halfwidth_all[condname][isess] = np.nan
            # maxtimepoint_all[condname][isess] = np.nan
        
# box plot 
fig, axs = plt.subplots(1,2)
fig.set_figheight(5)
fig.set_figwidth(5*2)

# subplot 1 - half max width - all animals
halfwidth_all_df = pd.DataFrame.from_dict(halfwidth_all,orient='index')
halfwidth_all_df = halfwidth_all_df.transpose()
halfwidth_all_df['type'] = 'all'
#
df_long=pd.concat([halfwidth_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=conditions,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.boxplot(ax=axs[0],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[0],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[0].set_xlabel('')
axs[0].set_xticklabels(conditions)
axs[0].xaxis.set_tick_params(labelsize=15,rotation=45)
axs[0].set_ylabel("half max width",fontsize=15)
axs[0].set_title('all animals' ,fontsize=24)
axs[0].set_ylim([0,10])
axs[0].legend(fontsize=18)


# subplot 2 - max time point - all animals
maxtimepoint_all_df = pd.DataFrame.from_dict(maxtimepoint_all,orient='index')
maxtimepoint_all_df = maxtimepoint_all_df.transpose()
maxtimepoint_all_df['type'] = 'all'
#
df_long=pd.concat([maxtimepoint_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=conditions,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.boxplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type',
#                   alpha=.9,size= 9,dodge=True,legend=False)
axs[1].set_xlabel('')
axs[1].set_xticklabels(conditions)
axs[1].xaxis.set_tick_params(labelsize=15,rotation=45)
axs[1].set_ylabel("max time point",fontsize=15)
axs[1].set_title('all animals' ,fontsize=24)
axs[1].set_ylim([-4,4])
axs[1].legend(fontsize=18)


savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)

    plt.savefig(figsavefolder+"socialgaze_distribution_summaryplot_halfmaxWitdh.pdf")



In [ ]:
halfwidth_all_df = pd.DataFrame.from_dict(halfwidth_all,orient='index')
halfwidth_all_df = halfwidth_all_df.transpose()
halfwidth_all_df['type'] = 'all'
#
df_long=pd.concat([halfwidth_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=conditions,var_name='condition', value_name='value')
df_long2 = df_long2[~np.isnan(df_long2['value'])]    
#    
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#
# anova
cw_lm=ols('value ~ condition', data=df_long2).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))
# post hoc test 
tukey = pairwise_tukeyhsd(endog=df_long2['value'], groups=df_long2['condition'], alpha=0.05)
print(tukey)

In [ ]:
maxtimepoint_all_df = pd.DataFrame.from_dict(maxtimepoint_all,orient='index')
maxtimepoint_all_df = maxtimepoint_all_df.transpose()
maxtimepoint_all_df['type'] = 'all'
#
df_long=pd.concat([maxtimepoint_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=conditions,var_name='condition', value_name='value')
df_long2 = df_long2[~np.isnan(df_long2['value'])]    
#    
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#
# anova
cw_lm=ols('value ~ condition', data=df_long2).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))
# post hoc test 
tukey = pairwise_tukeyhsd(endog=df_long2['value'], groups=df_long2['condition'], alpha=0.05)
print(tukey)

####  plot the gaze distribution along the continuous variables - pool across animals  and sesisons
#### only focus on "otherani_otherlever_dist", plot the ratio of number of 'decrasing' phase vs 'increasing' phase

In [ ]:
animal1_fixedorders = ['eddie','dodson','dannon','ginger',]
animal2_fixedorders = ['sparkle','scorch','kanga_1','kanga_2',]
nanimalpairs = np.shape(animal1_fixedorders)[0]

type_tgt = 'otherani_otherlever_dist'
# type_tgt = 'animal_lever_dist'
# type_tgt = 'animal_animal_dist'

# grouptypes = ['self reward','1s threshold','animal1 autolever','animal2_autolever']
grouptypes = ['self reward','1s threshold','self autolever','other autolever']
coopthres_IDs = [0, 1, 3.5, 4]
ngrouptypes = np.shape(grouptypes)[0]

gazeDist_foreachgroup_foreachAni = dict.fromkeys(grouptypes,[])
gazeDist_foreachgroup_all = dict.fromkeys(grouptypes,[])

malenames = ['eddie','dodson','dannon',]
femalenames = ['sparkle','scorch','kanga_1','kanga_2','ginger',]
gazeDist_foreachgroup_male = dict.fromkeys(grouptypes,[])
gazeDist_foreachgroup_female = dict.fromkeys(grouptypes,[])

subnames = ['eddie','dodson','dannon','ginger',]
domnames = ['sparkle','scorch','kanga_1','kanga_2',]
gazeDist_foreachgroup_sub = dict.fromkeys(grouptypes,[])
gazeDist_foreachgroup_dom = dict.fromkeys(grouptypes,[])

#
for igrouptype in np.arange(0,ngrouptypes,1):
    
    grouptype = grouptypes[igrouptype]
    coopthres_ID = coopthres_IDs[igrouptype]
    
    gazeDist_foreachgroup_foreachAni[grouptype] = dict.fromkeys(animal1_fixedorders+animal2_fixedorders,[])
    
    #
    for ianimalpair in np.arange(0,nanimalpairs,1):
        animal1 = animal1_fixedorders[ianimalpair]
        animal2 = animal2_fixedorders[ianimalpair]
        
        if (animal2 == 'kanga_1') | (animal2 == 'kanga_2'):
            animal2_filename = 'kanga'
        else:
            animal2_filename = animal2

        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1+animal2_filename+'/'
        
        with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            tasktypes_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            coopthres_all_dates = pickle.load(f)
        with open(data_saved_subfolder+'/gazeDist_phaseof_contbhvvar_all_dates_'+animal1+animal2_filename+'.pkl', 'rb') as f:
            gazeDist_phaseof_contbhvvar_all_dates = pickle.load(f) 
            
        #
        # 100: self; 3: 3s coop; 2: 2s coop; 1.5: 1.5s coop; 1: 1s coop; -1: no-vision
        tasktypes_all_dates[tasktypes_all_dates==5] = -1 # change the task type code for no-vision
        coopthres_forsort = (tasktypes_all_dates-1)*coopthres_all_dates/2
        coopthres_forsort[coopthres_forsort==0] = 0 # get the cooperation threshold for sorting
        
        # reorganize to separate the session into self autolever and other autolever
        if grouptype == 'self autolever':
             
            # for animal 1
            dates_all_dates = np.array(list(gazeDist_phaseof_contbhvvar_all_dates.keys()))
            dates_igrouptype = dates_all_dates[coopthres_forsort.transpose()[0]==3.5]
            ndates = np.shape(dates_igrouptype)[0]
            #
            gazeDist_ratios_animal1 = np.ones(np.shape(dates_igrouptype))*np.nan

            for idate in np.arange(0,ndates,1):     
                #
                gazeDist_idate = gazeDist_phaseof_contbhvvar_all_dates[dates_igrouptype[idate]][animal1][type_tgt]
                gateDist_ratio_idate = np.sum(gazeDist_idate['phase']=='decreasing')/np.sum(gazeDist_idate['phase']=='increasing')
                gazeDist_ratios_animal1[idate] = gateDist_ratio_idate
            # 
            gazeDist_foreachgroup_foreachAni[grouptype][animal1] = gazeDist_ratios_animal1

            # for animal 2
            dates_all_dates = np.array(list(gazeDist_phaseof_contbhvvar_all_dates.keys()))
            dates_igrouptype = dates_all_dates[coopthres_forsort.transpose()[0]==4]
            ndates = np.shape(dates_igrouptype)[0]
            #
            gazeDist_ratios_animal2 = np.ones(np.shape(dates_igrouptype))*np.nan

            for idate in np.arange(0,ndates,1):     
                #
                gazeDist_idate = gazeDist_phaseof_contbhvvar_all_dates[dates_igrouptype[idate]][animal2_filename][type_tgt]
                gateDist_ratio_idate = np.sum(gazeDist_idate['phase']=='decreasing')/np.sum(gazeDist_idate['phase']=='increasing')
                gazeDist_ratios_animal2[idate] = gateDist_ratio_idate
            # 
            gazeDist_foreachgroup_foreachAni[grouptype][animal2] = gazeDist_ratios_animal2
        
        elif grouptype == 'other autolever':
            
            # for animal 1
            dates_all_dates = np.array(list(gazeDist_phaseof_contbhvvar_all_dates.keys()))
            dates_igrouptype = dates_all_dates[coopthres_forsort.transpose()[0]==4]
            ndates = np.shape(dates_igrouptype)[0]
            #
            gazeDist_ratios_animal1 = np.ones(np.shape(dates_igrouptype))*np.nan

            for idate in np.arange(0,ndates,1):     
                #
                gazeDist_idate = gazeDist_phaseof_contbhvvar_all_dates[dates_igrouptype[idate]][animal1][type_tgt]
                gateDist_ratio_idate = np.sum(gazeDist_idate['phase']=='decreasing')/np.sum(gazeDist_idate['phase']=='increasing')
                gazeDist_ratios_animal1[idate] = gateDist_ratio_idate
            # 
            gazeDist_foreachgroup_foreachAni[grouptype][animal1] = gazeDist_ratios_animal1

            # for animal 2
            dates_all_dates = np.array(list(gazeDist_phaseof_contbhvvar_all_dates.keys()))
            dates_igrouptype = dates_all_dates[coopthres_forsort.transpose()[0]==3.5]
            ndates = np.shape(dates_igrouptype)[0]
            #
            gazeDist_ratios_animal2 = np.ones(np.shape(dates_igrouptype))*np.nan

            for idate in np.arange(0,ndates,1):     
                #
                gazeDist_idate = gazeDist_phaseof_contbhvvar_all_dates[dates_igrouptype[idate]][animal2_filename][type_tgt]
                gateDist_ratio_idate = np.sum(gazeDist_idate['phase']=='decreasing')/np.sum(gazeDist_idate['phase']=='increasing')
                gazeDist_ratios_animal2[idate] = gateDist_ratio_idate
            # 
            gazeDist_foreachgroup_foreachAni[grouptype][animal2] = gazeDist_ratios_animal2
            
        else:
            dates_all_dates = np.array(list(gazeDist_phaseof_contbhvvar_all_dates.keys()))
            dates_igrouptype = dates_all_dates[coopthres_forsort.transpose()[0]==coopthres_ID]
            ndates = np.shape(dates_igrouptype)[0]
            #
            gazeDist_ratios_animal1 = np.ones(np.shape(dates_igrouptype))*np.nan
            gazeDist_ratios_animal2 = np.ones(np.shape(dates_igrouptype))*np.nan

            for idate in np.arange(0,ndates,1):     
                # 
                # for animal 1
                gazeDist_idate = gazeDist_phaseof_contbhvvar_all_dates[dates_igrouptype[idate]][animal1][type_tgt]
                gateDist_ratio_idate = np.sum(gazeDist_idate['phase']=='decreasing')/np.sum(gazeDist_idate['phase']=='increasing')
                gazeDist_ratios_animal1[idate] = gateDist_ratio_idate
                # for animal 2
                gazeDist_idate = gazeDist_phaseof_contbhvvar_all_dates[dates_igrouptype[idate]][animal2_filename][type_tgt]
                gateDist_ratio_idate = np.sum(gazeDist_idate['phase']=='decreasing')/np.sum(gazeDist_idate['phase']=='increasing')
                gazeDist_ratios_animal2[idate] = gateDist_ratio_idate
            # 
            gazeDist_foreachgroup_foreachAni[grouptype][animal1] = gazeDist_ratios_animal1
            gazeDist_foreachgroup_foreachAni[grouptype][animal2] = gazeDist_ratios_animal2
            
            
    # combine across all animals
    gazeDist_foreachgroup_all[grouptype] = np.hstack(list(gazeDist_foreachgroup_foreachAni[grouptype].values()))
    
    # combine across male and female
    df = pd.DataFrame.from_dict(gazeDist_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[malenames]
    gazeDist_foreachgroup_male[grouptype] = df.values.ravel()
    #
    df = pd.DataFrame.from_dict(gazeDist_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[femalenames]
    gazeDist_foreachgroup_female[grouptype] = df.values.ravel()
    
    # combine across sub and dom
    df = pd.DataFrame.from_dict(gazeDist_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[subnames]
    gazeDist_foreachgroup_sub[grouptype] = df.values.ravel()
    #
    df = pd.DataFrame.from_dict(gazeDist_foreachgroup_foreachAni[grouptype],orient='index')
    df = df.transpose()[domnames]
    gazeDist_foreachgroup_dom[grouptype] = df.values.ravel()
    
    
        
# box plot 
fig, axs = plt.subplots(3,1)
fig.set_figheight(5*3)
fig.set_figwidth(3*2)

# subplot 1 - all animals
gazeDist_foreachgroup_all_df = pd.DataFrame.from_dict(gazeDist_foreachgroup_all,orient='index')
gazeDist_foreachgroup_all_df = gazeDist_foreachgroup_all_df.transpose()
gazeDist_foreachgroup_all_df['type'] = 'all'
#
df_long=pd.concat([gazeDist_foreachgroup_all_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.boxplot(ax=axs[0],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[0],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[0].plot([-0.5,3.5],[1,1],'k--')
axs[0].set_xlabel('')
axs[0].set_xticklabels('')
axs[0].xaxis.set_tick_params(labelsize=15)
axs[0].set_ylabel("ratio",fontsize=15)
axs[0].set_title("social gaze on the decreasing phase / inecreasing phase of "+type_tgt)
axs[0].set_ylim([-0.1,4.7])
axs[0].legend(fontsize=18)

# subplot 2 - male and female animals
gazeDist_foreachgroup_male_df = pd.DataFrame.from_dict(gazeDist_foreachgroup_male,orient='index')
gazeDist_foreachgroup_male_df = gazeDist_foreachgroup_male_df.transpose()
gazeDist_foreachgroup_male_df['type'] = 'male'
gazeDist_foreachgroup_female_df = pd.DataFrame.from_dict(gazeDist_foreachgroup_female,orient='index')
gazeDist_foreachgroup_female_df = gazeDist_foreachgroup_female_df.transpose()
gazeDist_foreachgroup_female_df['type'] = 'female'
#
df_long=pd.concat([gazeDist_foreachgroup_male_df,gazeDist_foreachgroup_female_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.boxplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[1],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[1].plot([-0.5,3.5],[1,1],'k--')
axs[1].set_xlabel('')
axs[1].set_xticklabels('')
axs[1].xaxis.set_tick_params(labelsize=15)
axs[1].set_ylabel("ratio",fontsize=15)
# axs[1].set_title('male and female' ,fontsize=24)
axs[1].set_ylim([-0.1,4.7])
axs[1].legend(fontsize=18)

# subplot 3 - dom and sub animals
gazeDist_foreachgroup_sub_df = pd.DataFrame.from_dict(gazeDist_foreachgroup_sub,orient='index')
gazeDist_foreachgroup_sub_df = gazeDist_foreachgroup_sub_df.transpose()
gazeDist_foreachgroup_sub_df['type'] = 'sub'
gazeDist_foreachgroup_dom_df = pd.DataFrame.from_dict(gazeDist_foreachgroup_dom,orient='index')
gazeDist_foreachgroup_dom_df = gazeDist_foreachgroup_dom_df.transpose()
gazeDist_foreachgroup_dom_df['type'] = 'dom'
#
df_long=pd.concat([gazeDist_foreachgroup_sub_df,gazeDist_foreachgroup_dom_df])
df_long2 = df_long.melt(id_vars=['type'], value_vars=grouptypes,var_name='condition', value_name='value')
# 
# barplot ans swarmplot
seaborn.boxplot(ax=axs[2],data=df_long2,x='condition',y='value',hue='type')
# seaborn.swarmplot(ax=axs[2],data=df_long2,x='condition',y='value',hue='type',alpha=.9,size= 9,dodge=True,legend=False)
axs[2].plot([-0.5,3.5],[1,1],'k--')
axs[2].set_xlabel('')
axs[2].set_xticklabels(axs[2].get_xticklabels(),rotation=45)
axs[2].xaxis.set_tick_params(labelsize=15)
axs[2].set_ylabel("ratio",fontsize=15)
# axs[2].set_title('sub and dom' ,fontsize=24)
axs[2].set_ylim([-0.1,4.7])
axs[2].legend(fontsize=18)

        
savefigs = 1
if savefigs:
    figsavefolder = data_saved_folder+'figs_for_3LagDBN_and_bhv_singlecam_wholebodylabels_allsessions_basicEvents_autolever/'+savefile_sufix+'/'+cameraID+'/autolever/'
    if not os.path.exists(figsavefolder):
        os.makedirs(figsavefolder)
    plt.savefig(figsavefolder+"averaged_gazenumberratio_decreasingphaseVSincreasingphase"+type_tgt+"_"+'acrossAllAnimals.pdf')
            
    
        

In [ ]:
import scipy.stats as st

xxx = gazeDist_foreachgroup_all_df['1s threshold']
# xxx = gazeDist_foreachgroup_dom_df['1s threshold']
# xxx = gazeDist_foreachgroup_female_df['1s threshold']
xxx_clean = xxx[~np.isnan(xxx) & ~np.isinf(xxx)]

st.ttest_1samp(xxx_clean,1)